**Backburner:**
1. Openings (see April 25 notes.)
2. Chess manual rule-based reading (see April 25 notes.)
3. Have the log write its output to a textfile (not needed yet.)

**Things to do:**
1. car-step summarization: once you have all the scores calculated, you have more insight to give a summarization. This will create a trend and you will be able to say whether you're winning or losing. 
1. Turning points. Part of a longer explanation between moves. While a certain score is rising, that’s all part of the same thing. When the score stops rising, that is a turning point. With car-step summarization, you can do this.
1. Even when following an opening, should the agent decide to quit it, if a non-opening move has a much higher score than the opening move?
1. An agent's thinking across multiple moves created and saved to the Log. Even just a general strategy like: "Defending" or "Attacking" could work. Some scores are defensive (like safety or territory) or offensive (like territory or mobility). Observing the trends between these.

**Things done:**
1. genetic algorithm implemented
1. sanity/overfitting check
1. in progress: add more games to dataset (currently: 25 games, ~50 moves each)
1. save position dicts for all legal moves as part of the fitness set. Should be faster this way.




In [ ]:
import multiprocessing
import time

start_time = time.time()
def spawn(num):
  while num < 1000000:
    num = num + 1

jobs = []
for i in range(10):
  p = multiprocessing.Process(target=spawn, args=(i,))
  jobs.append(p)
  p.start()

for job in jobs:
  job.join()

print("multiprocess version:")
print("--- %s seconds ---" % (time.time() - start_time))

start_time2 = time.time()
for i in range(10):
  num = i
  while num < 1000000:
    num = num + 1

print("singleprocess version:")
print("--- %s seconds ---" % (time.time() - start_time2))

multiprocess version:
--- 1.023183822631836 seconds ---
singleprocess version:
--- 1.1408798694610596 seconds ---


In [ ]:
!pip install python-chess --upgrade 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 148 kB 6.9 MB/s 


In [ ]:
!pip install --force-reinstall chess 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached chess-1.9.3-py3-none-any.whl (148 kB)
  Attempting uninstall: chess
    Found existing installation: chess 1.9.3
    Uninstalling chess-1.9.3:
      Successfully uninstalled chess-1.9.3


In [ ]:
!pip install stockfish

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#https://colab.research.google.com/drive/1Xk9MibJ9Fli5tIlDvo88hcZrI76rqZN5#scrollTo=YUSXjfAvDJ6D
#(lol I think it was someone's homework - of course we're just borrowing their setup)
!wget https://www.dropbox.com/sh/75gzfgu7qo94pvh/AACk_w5M94GTwwhSItCqsemoa/Stockfish%205/stockfish-5-linux.zip
!unzip stockfish-5-linux.zip
!chmod +x stockfish-5-linux/Linux/stockfish_14053109_x64

In [ ]:
# Important for chess
import random
import chess
import chess.engine
import chess.svg
from stockfish import Stockfish #https://pypi.org/project/stockfish/
import time
import os
import IPython
from IPython.display import SVG, display, HTML, clear_output

pieceNames = {
  'k':'(Black King)',
  'q':'(Black Queen)',
  'r':'(Black Rook)',
  'b':'(Black Bishop)',
  'n':'(Black Knight)',
  'p':'(Black Pawn)',
  'K':'(White King)',
  'Q':'(White Queen)',
  'R':'(White Rook)',
  'B':'(White Bishop)',
  'N':'(White Knight)',
  'P':'(White Pawn)'
}

In [ ]:
class Log:
    def __init__(self):
        """
        The log is responsible for keeping a record of the game and then writing it to a text file afterwards.
        It keeps a list of all moves made during the game, in order. The list has elements of the form: (turn number, player, move in uci, ai's explanation)
        It should also have a function to look up a move+board (taken in uci notation) and give general analysis of that move.
        """
        self.record = []
    def log_move(self, turn, player, colour, uci, active_explanation, opponent_analysis):
        self.record.append((turn, player, colour, uci, active_explanation, opponent_analysis))

    def print_log(self):
        """
        Prints log to terminal.
        """
        # self.record is a list of tuples: (turn, player, colour, move_uci, explanation, opp_explanation)
        for (turn, player, colour, uci, explanation, opp_explanation) in self.record:
            # e.g.: on turn 1, white played e2e4 | It's the best move in the game.
            print(f"On move {turn}, {str(player)} ({colour}) played {uci} with explanation: {explanation}. Their opponent commented: {opp_explanation}")

    def write_log(self):
        """
        Creates and writes the game record to a text file, stored in the local drive folder.
        """
        pass

    def look_up_move(self, board, move):
        """
        Looks up move and gives general analysis.
        In what form? A score int? an explanation string?
        """
        pass


In [ ]:
"""
Responsible for qualitative description of moves.
"""
def monitor_move(board, start, to, current_piece, agent, captured_piece, promoted, in_check):
    symbol = chess.UNICODE_PIECE_SYMBOLS[current_piece.symbol()]
    print("Piece summary for " + symbol + " " + pieceNames[current_piece.symbol()])
    print("UCI: " + start + to)

    """
    First, qual_description summarizes the move made.
    Then, the agent explains why it made that move.
    """
    print("=====Qualitative Description=====")
    ##qual_description(start, to, current_piece, captured_piece, promoted)
    qual_description(start, to, current_piece, captured_piece, promoted, in_check)

    agent.generate_explanation(current_piece)
    print("\n=====Active Agent Explanation=====\n(" + agent.get_name() + ")\n" + agent.get_explanation())

def qual_description(start, finish, piece, captured_piece, promoted, in_check):
    """
    This function describes what happened in the last move.
    (It does not offer any explanation, the agent does that.)
    Important elements of qualitative description:
    -towards opposing/own side
    -captured an opposing piece and what type
    -promoted
    -put opponent into check
    """

    def exp_movement():
        # Set the operator for black or white.
        print("moved from", start, "to", finish)
        start_number = (start[1])
        end_number = (finish[1])
        # print("start/end = " + start_number + " / " + end_number)

        # moved towards enemy side/ally side/did not advance.
        if piece.color:  # if white 
            if (end_number > start_number):  # e.g. e2->e4
                print("Moved toward opposing side.")
            elif (end_number < start_number):
                print("Moved toward own side.")
        else:
            if (end_number < start_number):  # e.g. e7->e5
                print("Moved toward opposing side.")
            elif (end_number > start_number):
                print("Moved toward own side.")

    def exp_capture(captured_piece):
        """
        If there were more opponent pieces than there are now before this move, then the move captured a piece.
        """
        if captured_piece is not None:
            symbol = chess.UNICODE_PIECE_SYMBOLS[captured_piece.symbol()]
            print("Captured " + pieceNames[captured_piece.symbol()])
    def exp_promoted(promoted):
        if promoted:
            print("Promoted")

    def exp_check(in_check):
        if in_check:
            print("Check")
  
    exp_movement()
    exp_capture(captured_piece)
    exp_promoted(promoted)
    exp_check(in_check)


In [ ]:
"""
Notes:
  -how to record openings? With our example opening, (king's pawn openning), we encode the moves, which the agent then just uses directly.
    -however, this raises some issues: this encoding of the opening is only valid for the white player. Should openings be encoded both ways to allow both players to call upon them? That's the simplest way. Probably isn't a better way.
  -how should black react to white's openings?
    -look up theory to see what the black theory.
    -try to guess what
  -terminology: a chess opening is considered a 'system' when the same moves can be played for almost all of black's responses.
    -example: London System
    https://www.chess.com/openings/London-System
    -there are different variations too. There's reasoning behind

-black tries to guess white's opening, and pick a defense to counter that opening.
 -then white picks a counter-opening opening

"""
openings = {
    "mobility_test_w" : ["e2e3", "b1b3", "d1d4"],
    "mobility_test_b" : ["e7e5", "d8f6", "e5e4"],
    "check_castling_w1" : ["d2d4", "g1f3", "c1f4", "b1c3", "d1d2", "e2e4", "f1b5"],
    "check_castling_b1" : ["e7e5", "d8g5", "d7d6", "b8c6", "c8e6", "f8e7", "g8f6"],
    "bongcloud" : ["e4e5", "e1e2"],
    "london_system" : ["d2d4", "g1f3", "c1f4"],
    "london_mainline" : ["d2d4", "c1f4", "e2e3", "c1c3", "b1d2", "g1f3"],
    "london_indian_setup" : ["d2d4", "f1f4", "g1g3", "e2e3"],
    "jobava_london" : ["d2d4", "b1c3", "c1f4"]
}

def get_opening(color, opening_str):
    """
    Helper function, used by agents to get an opening.
    Returns the sequence of moves by the color given.

    If it doesn't recognize the opening_str argument, then it returns an empty list, which the agent interprets as no opening.
    """
    # print(opening_str)
    # print(openings)
    if opening_str in openings:
        # get the right-coloured version of it
        return openings[opening_str].copy()
    else:
        return []

In [ ]:
"""
A useful function for displaying the color of a player
"""
def who(player):
    return "White" if player == chess.WHITE else "Black"

"""
A useful function for displaying the color of a piece

Constants for the side to move or the color of a piece.
chess.WHITE = 0
chess.BLACK = 1
"""
def which(piece):
    return "white" if piece.color else "black"

"""
A function for displaying the board as text
or as an SVG image
"""
def display_board(board, use_svg):
    if use_svg:
        return board._repr_svg_()
    else:
        return "<pre>" + str(board) + "</pre>"

def show_ascii(board):
    print(chess.BaseBoard.unicode(board))

def play_game(player1, player2, TURN_MAX, visual="svg", pause=0.1, adv=False):
    """
    player1, player2: classes that hold chess agents. capable of choosing a move and justification
    visual: "simple" | "svg" | None
    """
    log = Log()
    use_svg = (visual == "svg")
    board = chess.Board()
    turns = 0

    try:
        while not board.is_game_over(claim_draw=True) and turns <= TURN_MAX:
            clear_output(wait=True)
            if board.turn == chess.WHITE:
                active_player = player1
                non_active_player = player2
            else:
                active_player = player2
                non_active_player = player1
            name = who(board.turn)

            # we save the active player's dict here, for the analysis. Needed for analyzing pre-move positions.
            save_active_dict = active_player.get_pieces_dict().copy()
            save_active_lastTurnScores = active_player.get_lastTurnScore()

            move = active_player.move_select(active_player.get_color(), False, turns / 2, board, active_player.get_pieces_dict(), non_active_player.get_pieces_dict(), save_active_lastTurnScores)
            uci = move.uci()
            print("move UCI early printout = " + uci)

            # save information about the piece we're about to move.
            # we need to save it here to give the explanation in sync with the board update.
            start = uci[0:2]
            end = uci[2::]
            board_index = chess.SQUARE_NAMES.index(start)
            acting_piece = board.piece_at(board_index)

            # non-active player makes their analysis
            # def interpret_opp_move(self, turn_num, board, opp_move, opponent_pieces_dict)
            opp_analysis = non_active_player.interpret_opp_move(turns / 2, board, move, save_active_dict, save_active_lastTurnScores)

            # save information for qual analysis (regarding promotion)
            promoted = False
            if len(end) > 2:
                promoted = True
                end = end[0:2]

            # save information for qual analysis. (Regarding captures)
            end_board_index = chess.SQUARE_NAMES.index(end)
            captured_piece = board.piece_at(end_board_index)
            if captured_piece is not None:
              non_active_player.remove_piece(non_active_player.get_pieces_dict(), end_board_index)
              
            # update the board/gamestate.
            board.push_uci(uci)

            # check if the move has caused check
            in_check = board.is_check()
            
            # updates visuals.
            if visual is not None:
                if visual == "svg":
                    html_code = chess.svg.board(board, size=300)
                    display(IPython.display.HTML(html_code))
                else:
                    show_ascii(board)

            # prints qualitative description and AI move explanation.
            #   new args: opp_piece_diff, captured_piece
            monitor_move(board, start, end, acting_piece, active_player, captured_piece, promoted, in_check)

            # prints non-active player's analysis:
            print(f"\n=====Non-Active Agent Explanation=====\n({non_active_player.get_name()})\n{opp_analysis}\n")

            # writes move to log.
            if turns % 2 == 0: 
              colour = "White"
            else:
              colour = "Black"
            if opp_analysis is not None:
                log.log_move(turns, active_player.get_name(), colour, start + end, active_player.get_explanation(), opp_analysis)
            turns+=1

            if adv is False:
                input()
            else:
                time.sleep(pause)

    except KeyboardInterrupt:
        msg = "Game interrupted!"
        return (None, msg, board)
    result = None
    if board.is_checkmate():
        msg = "checkmate: " + who(not board.turn) + " wins!"
        result = not board.turn
    elif board.is_stalemate():
        msg = "draw: stalemate"
    elif board.is_fivefold_repetition():
        msg = "draw: 5-fold repetition"
    elif board.is_insufficient_material():
        msg = "draw: insufficient material"
    elif board.can_claim_draw():
        msg = "draw: claim"
    elif turns > TURN_MAX:
        msg = "turn limit elapsed."
        log.print_log()
    """
    if visual is not None:
      # also ends here when moves are elapsed. 
      print(msg)  # This cases an error in the break
    """
    return (result, msg, board)


In [ ]:
class RandomAgent:
    def __init__(self, color):
        """
        Also the base agent.
        How does this agent work?
        on its turn, it selects a move randomly from all possible moves.
        It explains that it selected the move randomly.
        """
        self.bestScore = -1
        self.exp = "no move yet made."
        self.color = color
        self.init_pieces()
    def get_color(self):
        return self.color
    def init_pieces(self):
        if self.color: # if white
          self.piecesDictionary = {"p1": "a2","p2": "b2","p3": "c2","p4": "d2","p5": "e2","p6": "f2","p7": "g2","p8": "h2","r1": "a1","r2": "h1","n1": "b1","n2": "g1","b1": "c1","b2": "f1","q": "d1","k": "e1"}
        else: # if black
          self.piecesDictionary = {"p1": "a7","p2": "b7","p3": "c7","p4": "d7","p5": "e7","p6": "f7","p7": "g7","p8": "h7","r1": "a8","r2": "h8","n1": "b8","n2": "g8","b1": "c8","b2": "f8","q": "d8","k": "e8"}
    def remove_piece(self, this_dict, remove_pos):
        # Called when a piece is captured by the opponent. Removes the pair from the dictionary.
        # print(f"remove piece: {remove_pos}")
        for key, value in this_dict.items():
          if chess.SQUARE_NAMES.index(value) == remove_pos:
              # print("MATCH")
              del this_dict[key]
              # print("removed the piece. Mission clear, chewy!")
              # print(f"dict = {self.piecesDictionary}")
              break
    def update_pieces(self, this_dict, old_pos, new_pos):
        # Updates the entry in self.piecesDictionary for the relevant piece. Assumes old_pos and new_pos are exactly 2 characters each. E.g. a4
        # first, we have to find the key, which we can use old_pos to do.
        for key, value in this_dict.items():
          # print(f"update_pieces() comparison between {value} and {old_pos}")
          if value == old_pos:
              this_dict[key] = new_pos
              # print(this_dict)
              return
    def get_key_at_pos(self, this_dict, find_pos):
        for key, value in this_dict.items():
          # print(f"get_key_at_pos(). searching for={find_pos}. key={key}, value={value}")
          if value == find_pos:
              return key
    def get_pieces_dict(self):
        return self.piecesDictionary
    def get_name(self):
        return "RandomAgent"
    def get_explanation(self):
        return self.exp
    def generate_explanation(self, piece):
        self.exp = "Move was chosen randomly."
    def move_select(self, analysing_opp, turn_num, board, own_pieces_dict, opponent_pieces_dict):
        """
        Randomly select a legal move.
        Also, generate and save the move's explanation here.
        Finally, update the position of the moved piece in piecesDictionary.
        """
        move = random.choice(list(board.legal_moves))
        self.update_pieces(self.piecesDictionary, str(move)[0:2], str(move)[2:4])
        return move
    def interpret_opp_move(self, board, move, opponent_pieces_dict):
        return "... No idea."

class BuildableAgent(RandomAgent):
    def __init__(self, color, norm_method, openingChoice, mobilityWeight, mobilityWeightList, staticAnalysisWeight, safetyWeight, territoryWeight, fortifyWeight, kingsPawnShieldWeight, bishopPositionWeight, knightPositionWeight, hideKingTuple=(0,0), hideQueenTuple=(0,0)):
        """
        The buildable agent is the takes arguments that assign weights to its various scoring functions.
        When scoring or explaining moves, it uses all scoring methods that were not assigned a weight of zero.
        """
        self.bestScore = -1
        self.exp = "no move yet made."
        self.color = color
        self.norm_method = norm_method
        self.opening = self.opening_select(openingChoice)
        self.init_pieces()

        self.mobilityWeight = mobilityWeight
        self.mobilityWeightList = mobilityWeightList

        self.staticAnalysisWeight = staticAnalysisWeight
        self.safetyWeight = safetyWeight
        self.territoryWeight = territoryWeight
        self.fortifyWeight = fortifyWeight

        self.kingsPawnShieldWeight = kingsPawnShieldWeight
        self.bishopPositionWeight = bishopPositionWeight
        self.knightPositionWeight = knightPositionWeight
        
        # hide tuples: weight, turns until the agent stops caring
        self.hideKingTuple = hideKingTuple
        self.hideQueenTuple = hideQueenTuple

        # state
        self.hasCastled = False 
        self.lastTurnScore = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0] # save the (normalized) scoreList of the previous turn.
    def get_lastTurnScore(self):
        return self.lastTurnScore
    def get_name(self):
        """
        The name returned depends on the scoring function weights. Used for recording the agent's moves in the Log.
        """
        return f"BuildableAgent (mob: {self.mobilityWeight}, stat: {self.staticAnalysisWeight})"
    def generate_explanation(self, piece):
        """
        Generates an explanation based on the decomposition of its utility function and the move made.
        """
        return self.exp
    def opening_select(self, opening_str):
        """
        Retrieves a list of strings (move UCIs; i.e. the opening's moves) that will be played until finished or the agent is unable to continue playing it (e.g. because of check or a captured piece.)
        retrieves the correct opening based on opening_str and color.
        """
        # returns, for example, ["d2d4", "g1f3", "c1f4"]
        return get_opening(self.color, opening_str)

    def normalize(self, scoreList):
        # returns a normalized version of the scoreList, based on the norm_method argument.
        # normalization method:
        # 0: score / max_score

        # scoreList legend (by index):
        # 0: mobility score
        # 1: static analysis score
        # 2: hide king score
        # 3: hide queen score
        # 4: safety score
        # 5: territory score
        # 6: fortify score
        # 7: king's pawn shield
        # 8: bishop position score
        # 9: knight position score

        #note: the max score for the mobility one should be calculated based on self.mobilityWeightList

        if self.norm_method == 0:        
          scoreList = [scoreList[0] / 100, scoreList[1] / 41, scoreList[2] / 1, scoreList[3] / 1, scoreList[4] / 16, scoreList[5] / 16, scoreList[6] / 16, scoreList[7] / 4, scoreList[8] / 2, scoreList[9] / 2]

        return [round(score, 2) for score in scoreList]

    
    def handle_castling(self, some_dict, start, move_str, color):
            """
            Handle castling by updating the position of the rook in some_dict.
            Returns some_dict updated.
            If move was not castling, then does nothing.
            """ 
            # if not analysing_opp:
                # print(f"handle_castling(). some_dict={some_dict}")
                # print(f"handle_castling(). move_str={move_str}. start={start}. piece at start={self.get_key_at_pos(some_dict, start)}")
            # if we are moving the king
            if self.get_key_at_pos(some_dict, start) == "k":
                # if white
                if color:
                    # move was a castle if: piece at start pos is king, and uci is either: e1b1 or e1g1
                    if move_str == "e1g1":
                        self.update_pieces(some_dict, "h1", "f1")  # update rook, king will be updated normally.
                    elif move_str == "e1c1":
                        self.update_pieces(some_dict, "a1", "d1")  # update rook, king will be updated normally.
                # else if black
                else:
                    # move was a castle if: piece at staat pos is king, and uci is either: e8b8 or e8g8
                    if move_str == "e8g8":
                        self.update_pieces(some_dict, "h8", "f8")  # update rook, king will be updated normally.
                    elif move_str == "e8c8":
                        self.update_pieces(some_dict, "a8", "d8")  # update rook, king will be updated normally.
            return some_dict
    def handle_en_passant(self, my_dict, opp_dict, start, end, move_str, color, board):
            """
            Handle en passant by removing the captured pawn from opp_dict.
            # how to detect en passant:
                # if piece is pawn
                # if end pos is empty
                # if end pos is in a different column
                # to remove captured piece:
                # if piece in front of pawn (depending on colour) is not empty
                #   then remove piece from opponent's dict
            """
            # if we are moving a pawn AND the end tile position is empty AND the end pos is in a different column from the start pos (meaning the pawn moved diagonally)
            if str(self.get_key_at_pos(my_dict, start))[0] == "p" and board.piece_at(chess.SQUARE_NAMES.index(end)) is None and start[0] != end[0]:
                if color:
                    remove_pos = end[0] + str(int(end[1]) - 1)
                else:
                    remove_pos = end[0] + str(int(end[1]) + 1)

                self.remove_piece(opp_dict, chess.SQUARE_NAMES.index(remove_pos))
            return opp_dict 

    def score_move_dict(self, color, analysing_opp, turn_num, board, own_pieces_dict, opponent_pieces_dict, last_scoreList, movesScoresDict):
        """
        Helper to move select. Scores each move and returns move, score pairs.
        """
        
        def hideQueenRule_score(movesScoreDict):
            """
            Scores negative if the queen is past a certain horizontal stripe (depending on color).
            Also depending on turn number?
            """
            if self.hideQueenTuple[0] == 0 or self.hideQueenTuple[1] <= turn_num or "q" not in own_pieces_dict:
                return
            for move, scoreList in movesScoresDict.items():
                move_str = move.uci()
                start = str(move_str[0:2])
                end = str(move_str[2::])

                tempPieces = own_pieces_dict.copy()
                self.update_pieces(tempPieces, start, end)

                hideQueenScore = hideAnalysis(color, tempPieces["q"])
                movesScoresDict[move][3] = hideQueenScore * self.hideQueenTuple[0]
        def hideKingRule_score(movesScoreDict):
            """
            Scores negative if the king is past a certain row (depending on color).
            Also depending on turn number?
            """
            if self.hideKingTuple[0] == 0 or self.hideKingTuple[1] <= turn_num or "k" not in own_pieces_dict:
                return                

            for move, scoreList in movesScoresDict.items():
                move_str = move.uci()
                start = str(move_str[0:2])
                end = str(move_str[2::])

                tempPieces = own_pieces_dict.copy()
                self.update_pieces(tempPieces, start, end)

                hideKingScore = hideAnalysis(color, tempPieces["k"])
                movesScoresDict[move][2] = hideKingScore * self.hideKingTuple[0]             
        def mobility_score(movesScoresDict):
            if self.mobilityWeight == 0:
                return
            for move, scoreList in movesScoresDict.items():
                # print(f"scoring mobility. (castling/enpassant watch) move={move.uci()}")
                move_str = move.uci()
                #tempPieces = own_pieces_dict.copy()
                #opp_tempPieces = opponent_pieces_dict.copy()

                start = str(move_str[0:2])
                end = str(move_str[2::])

                # check and handle en passant/castling if they occured.
                """
                tempPieces = self.handle_castling(tempPieces, start, move_str, color)
                opp_tempPieces = self.handle_en_passant(tempPieces, opp_tempPieces, start, end, move_str, color, board)

                self.update_pieces(tempPieces, start, end)
                self.remove_piece(opp_tempPieces, chess.SQUARE_NAMES.index(end))
                """

                newboard = board.copy()
                newboard.push(move)

                # score based on mobility and then add (mobilityScore*mobilityWeight) to the move's overall score.
                # print(f"Move = {move_str}. Mob w = {positional_analysis(newboard, tempPieces, color)} | mob b = {positional_analysis(newboard, opp_tempPieces, not color)}")
                mobilityScore = (mobility_analysis(newboard, self.mobilityWeightList, color) - mobility_analysis(newboard, self.mobilityWeightList, not color)) * self.mobilityWeight
                movesScoresDict[move][0] = mobilityScore
        def staticAnalysis_score(movesScoresDict):
            if self.staticAnalysisWeight == 0:
                return
            for move, scoreList in movesScoresDict.items():
                newboard = board.copy()
                newboard.push(move)
                staticAnalysisScore = staticAnalysis(newboard, color) - staticAnalysis(newboard, not color)
                movesScoresDict[move][1] = (staticAnalysisScore * self.staticAnalysisWeight)
        def safety_score(movesScoresDict):
            if self.safetyWeight == 0:
                return
            for move, scoreList in movesScoresDict.items():
                move_str = move.uci()
                tempPieces = own_pieces_dict.copy()
                opp_tempPieces = opponent_pieces_dict.copy()

                start = str(move_str[0:2])
                end = str(move_str[2::])

                # check and handle en passant/castling if they occured.
                tempPieces = self.handle_castling(tempPieces, start, move_str, color)
                opp_tempPieces = self.handle_en_passant(tempPieces, opp_tempPieces, start, end, move_str, color, board)

                self.update_pieces(tempPieces, start, end)
                self.remove_piece(opp_tempPieces, chess.SQUARE_NAMES.index(end))

                # -perform the move under consideration on a copy board.
                newboard = board.copy()
                newboard.push(move)
                          
                # safety_analysis(board, pieces_dict)
                # due to its essential nature, it is (little different from a criminal, and also) only calculated for one side. It's safety, not relative safety.
                safetyScore = safety_analysis(newboard, tempPieces) * self.safetyWeight
                movesScoresDict[move][4] = safetyScore
        def territory_score(movesScoresDict):           
            if self.territoryWeight == 0:
                return
            for move, scoreList in movesScoresDict.items():
                move_str = move.uci()

                start = str(move_str[0:2])
                end = str(move_str[2::])

                # -perform the move under consideration on a copy board.
                newboard = board.copy()
                newboard.push(move)

                territoryScore_own = territory_analysis(newboard)
                territoryScore_opp = territory_analysis(newboard)
                movesScoresDict[move][5] = (territoryScore_own - territoryScore_opp) * self.territoryWeight
                # print(f"territory score = {(territoryScore_own - territoryScore_opp) * self.territoryWeight}")
        def fortify_score(movesScoresDict):
            if self.fortifyWeight == 0:
                return
            for move, scoreList in movesScoresDict.items():
                move_str = move.uci()
                tempPieces = own_pieces_dict.copy()
                opp_tempPieces = opponent_pieces_dict.copy()

                start = str(move_str[0:2])
                end = str(move_str[2::])

                # check and handle en passant/castling if they occured.
                tempPieces = self.handle_castling(tempPieces, start, move_str, color)
                opp_tempPieces = self.handle_en_passant(tempPieces, opp_tempPieces, start, end, move_str, color, board)

                self.update_pieces(tempPieces, start, end)
                self.remove_piece(opp_tempPieces, chess.SQUARE_NAMES.index(end))

                # -perform the move under consideration on a copy board.
                newboard = board.copy()
                newboard.push(move)

                # fortify_analysis(board, piecesDict, color)

                fortify_own = fortify_analysis(newboard, tempPieces, color)
                # fortify_opp = fortify_analysis(newboard, opp_tempPieces, not color)
                
                #movesScoresDict[move][6] = (fortify_own - fortify_opp) * self.fortifyWeight
                movesScoresDict[move][6] = (fortify_own) * self.fortifyWeight
        def kingPawnShield_score(movesScoresDict):
            if self.kingsPawnShieldWeight == 0:
                return
            for move, scoreList in movesScoresDict.items():
                # print(f"scoring mobility. (castling/enpassant watch) move={move.uci()}")
                move_str = move.uci()
                tempPieces = own_pieces_dict.copy()
                opp_tempPieces = opponent_pieces_dict.copy()

                start = str(move_str[0:2])
                end = str(move_str[2::])

                # check and handle en passant/castling if they occured.
                tempPieces = self.handle_castling(tempPieces, start, move_str, color)
                opp_tempPieces = self.handle_en_passant(tempPieces, opp_tempPieces, start, end, move_str, color, board)

                self.update_pieces(tempPieces, start, end)
                self.remove_piece(opp_tempPieces, chess.SQUARE_NAMES.index(end))

                shieldScore = kings_pawn_shield(tempPieces) * self.kingsPawnShieldWeight
                movesScoresDict[move][7] = shieldScore
        def bishopPosition_score(movesScoresDict):
            if self.bishopPositionWeight == 0:
                return
            for move, scoreList in movesScoresDict.items():
                # print(f"scoring mobility. (castling/enpassant watch) move={move.uci()}")
                move_str = move.uci()
                tempPieces = own_pieces_dict.copy()
                opp_tempPieces = opponent_pieces_dict.copy()

                start = str(move_str[0:2])
                end = str(move_str[2::])

                # check and handle en passant/castling if they occured.
                tempPieces = self.handle_castling(tempPieces, start, move_str, color)
                opp_tempPieces = self.handle_en_passant(tempPieces, opp_tempPieces, start, end, move_str, color, board)

                self.update_pieces(tempPieces, start, end)
                self.remove_piece(opp_tempPieces, chess.SQUARE_NAMES.index(end))

                bishopScore = bishop_positions(tempPieces) * self.bishopPositionWeight
                movesScoresDict[move][8] = bishopScore
        def knightPosition_score(movesScoresDict):
            if self.knightPositionWeight == 0:
                return
            for move, scoreList in movesScoresDict.items():
                # print(f"scoring mobility. (castling/enpassant watch) move={move.uci()}")
                move_str = move.uci()
                tempPieces = own_pieces_dict.copy()
                opp_tempPieces = opponent_pieces_dict.copy()

                start = str(move_str[0:2])
                end = str(move_str[2::])

                # check and handle en passant/castling if they occured.
                tempPieces = self.handle_castling(tempPieces, start, move_str, color)
                opp_tempPieces = self.handle_en_passant(tempPieces, opp_tempPieces, start, end, move_str, color, board)

                self.update_pieces(tempPieces, start, end)
                self.remove_piece(opp_tempPieces, chess.SQUARE_NAMES.index(end))

                knightScore = knight_positions(tempPieces) * self.knightPositionWeight
                movesScoresDict[move][9] = knightScore

        """
        Rewrite scoring portion so it scores each move individually.
        for each move: score(move)
        """

        # scoring functions
        mobility_score(movesScoresDict)
        staticAnalysis_score(movesScoresDict)
        safety_score(movesScoresDict)
        territory_score(movesScoresDict)
        fortify_score(movesScoresDict)
        kingPawnShield_score(movesScoresDict)
        bishopPosition_score(movesScoresDict)
        knightPosition_score(movesScoresDict)

        # rule-based scoring
        hideKingRule_score(movesScoresDict)
        hideQueenRule_score(movesScoresDict)

        return movesScoresDict

    def move_select(self, color, analysing_opp, turn_num, board, own_pieces_dict, opponent_pieces_dict, last_scoreList):
        """
        First, attempts to follow its registered opening. This may fail if an opening move is not legal.
        analysing_opp is true when analysing an opponent's move.
        """
        legal_moves = list(board.legal_moves)
        if not analysing_opp and self.opening:
            # then try to follow our saved opening plan. Pop the front entry in the list.
            next_opening_move = self.opening.pop(0)
            # print(f"popped {next_opening_move} from opening. Remaining: {self.opening}")
            
            if next_opening_move in [str(move.uci()) for move in legal_moves]:
                # then that move is valid. Do the following:
                # -update piece dict
                # -generate explanation
                # -return move
                start = next_opening_move[0:2]
                end = next_opening_move[2::]

                self.piecesDictionary = self.handle_castling(self.piecesDictionary, start, next_opening_move, color)
                opponent_pieces_dict = self.handle_en_passant(self.piecesDictionary, opponent_pieces_dict, start, end, next_opening_move, color, board)
                self.update_pieces(self.piecesDictionary, start, end)

                self.exp = "I'm following an opening and placing my trust in those who have come before me."

                # print("before returning my chosen move:")
                # print(f"self.piecesDictionary={self.piecesDictionary}")
                return chess.Move.from_uci(next_opening_move)
            else:
                self.opening.clear()

        """
        Selects a move using all the scoring functions enabled.
        We create a dictionary pairing each of the legal moves to an int score. Each subscoring function then adds to that score.
        In the end, we return the move from the dictionary pair with the highest score.
        """
        movesScoresDict = {}
        for move in legal_moves:
          # print(f"considering move: {move.uci()}")
          # so we can break the scores down easier later, each score will be a list of scores, with each element corresponding to the score given by a certain scoring function. 

          # Legend (by index):
          # 0: mobility score
          # 1: static analysis score
          # 2: hide king score
          # 3: hide queen score
          # 4: safety score
          # 5: territory score
          # 6: fortify score
          movesScoresDict[move] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

        #score each move
        movesScoresDict = self.score_move_dict(color, analysing_opp, turn_num, board, own_pieces_dict, opponent_pieces_dict, last_scoreList, movesScoresDict)
        
        # return move with highest score: (randomly choose between all the best)
        bestMoves = []
        self.bestScore = -1
        for move, scoreList in movesScoresDict.items():
            score = sum(scoreList)
            if self.bestScore == -1 or score > self.bestScore:
              bestMoves = [(move, scoreList)]
              self.bestScore = score
            elif score == self.bestScore:
              bestMoves.append((move, scoreList))

        chosen_pair = random.choice(bestMoves)
        chosen_move = chosen_pair[0]

        chosen_scoreList = self.normalize(chosen_pair[1])
       
        # print(f"chosen_scoreList = {chosen_scoreList}")
        self.exp = f"I feel that this was the best of all legal moves, with a net score of {round(sum(chosen_scoreList), 2)}." + self.explain_move(turn_num, chosen_move, chosen_scoreList, last_scoreList)

        if not analysing_opp: 
            # print(f"chosen_scoreList = {chosen_scoreList}")

            self.piecesDictionary = self.handle_castling(self.piecesDictionary, str(chosen_move)[0:2], str(chosen_move), color)
            opponent_pieces_dict = self.handle_en_passant(self.piecesDictionary, opponent_pieces_dict, str(chosen_move)[0:2], str(chosen_move)[2:4], str(chosen_move), color, board)
            # print(f"1move chosen: opp dict after handling castle/en passant: {opponent_pieces_dict}")
            self.update_pieces(self.piecesDictionary, str(chosen_move)[0:2], str(chosen_move)[2:4])

            #print("before returning my chosen move:")
            #print(f"self.piecesDictionary={self.piecesDictionary}")

            self.lastTurnScore = chosen_scoreList
       
        return chosen_move
   
    def explain_move(self, turn_num, chosen_move, scoreList, lastTurnList):
        """
        Once our move has been chosen, then we explain it.
        break down scores into components. (e.g. netscore = 80. static score = 30x1.0. mob score = 100x0.5)

        Takes in a chosen move and its scorelist, returns a string.
        """
        retStr = "\nScore breakdown:"
        
        if self.mobilityWeight > 0.0:
            retStr += f"\n•Mobility: {lastTurnList[0]} -> {scoreList[0]} (weight={self.mobilityWeightList})"

        if self.staticAnalysisWeight > 0.0:
            retStr += f"\n•Static: {lastTurnList[1]} -> {scoreList[1]} (weight={self.staticAnalysisWeight})"

        if self.safetyWeight > 0.0:
            retStr += f"\n•Safety: {lastTurnList[4]} -> {scoreList[4]} (weight={self.safetyWeight})"

        if self.territoryWeight > 0.0:
            retStr += f"\n•Territory: {lastTurnList[5]} -> {scoreList[5]} (weight={self.territoryWeight})"
        
        if self.fortifyWeight > 0.0:
            retStr += f"\n•Fortify: {lastTurnList[6]} -> {scoreList[6]} (weight={self.fortifyWeight})"

        if self.kingsPawnShieldWeight > 0.0:
            retStr += f"\n•King's Pawn Shield: {lastTurnList[7]} -> {scoreList[7]} (weight={self.kingsPawnShieldWeight})"
        
        if self.bishopPositionWeight > 0.0:
            retStr += f"\n•Bishop's Pos: {lastTurnList[8]} -> {scoreList[8]} (weight={self.bishopPositionWeight})"

        if self.knightPositionWeight > 0.0:
            retStr += f"\n•Knight's Pos: {lastTurnList[9]} -> {scoreList[9]} (weight={self.knightPositionWeight})"

        # hide tuples: weight, turns until the agent stops caring
        if self.hideKingTuple[0] > 0.0 and self.hideKingTuple[1] >= turn_num:
            retStr += f"\n•Hide King: {lastTurnList[2]} -> {scoreList[2]} (weight={self.hideKingTuple[0]})"

        if self.hideQueenTuple[0] > 0.0 and self.hideQueenTuple[1] >= turn_num:
            retStr += f"\n•Hide Queen: {lastTurnList[3]} -> {scoreList[3]} (weight={self.hideQueenTuple[0]})"

        curSum = round(sum(scoreList), 2)
        lastSum = round(sum(lastTurnList), 2)
        retStr += f"\nNet: {lastSum} -> {curSum}\n"
        if curSum == lastSum:
            retStr += "The situation remains the same."
        elif curSum > lastSum:
            retStr += "The situation is improving."
        else:
            retStr += "The situation is deteriorating."

        return retStr

    def interpret_opp_move(self, turn_num, board, opp_move, opponent_pieces_dict, opp_last_scoreList):
        """
        Agent tries to come up with an explanation for the enemy's move.
        - pick move as if you were opponent (using your own scoring function)
        - compare your chosen move to enemy's chosen move.
        """

        """
        print(f"PRE-interpret().")
        print(f"own dict={self.piecesDictionary}")
        print(f"opp dict={opponent_pieces_dict}")
        html_code = chess.svg.board(board, size=300)
        display(IPython.display.HTML(html_code))
        """
        # decide best move from opponent's position, but using your scoring method:
        # def move_select(self, color, analysing_opp, turn_num, board, own_pieces_dict, opponent_pieces_dict)
        chosen_move = self.move_select(not self.color, True, turn_num, board, opponent_pieces_dict, self.piecesDictionary, opp_last_scoreList)

        # score the opponent's chosen move: (requires setting up the board and update opp_dict)
        newBoard = board.copy()
        newBoard.push_uci(opp_move.uci())
        move_str = str(opp_move.uci())
        self.update_pieces(opponent_pieces_dict, move_str[0:2], move_str[2::])

        build_opp_score = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        if self.mobilityWeight > 0.0:
            build_opp_score[0] = (mobility_analysis(newBoard, self.mobilityWeightList, not self.color) - mobility_analysis(newBoard, self.mobilityWeightList, self.color)) * self.mobilityWeight

        if self.staticAnalysisWeight > 0.0:
            build_opp_score[1] = (staticAnalysis(newBoard, not self.color) - staticAnalysis(newBoard, self.color)) * self.staticAnalysisWeight

        if self.safetyWeight > 0.0:
            build_opp_score[4] = safety_analysis(newBoard, self.piecesDictionary) * self.safetyWeight

        if self.territoryWeight > 0.0:
            new_newBoard = newBoard.copy()
            safetyScore_own = territory_analysis(new_newBoard)
            safetyScore_opp = territory_analysis(new_newBoard)
            build_opp_score[5] = (safetyScore_own - safetyScore_opp) * self.territoryWeight

        if self.fortifyWeight > 0.0:
            # print(f"fortify 1 = {fortify_analysis(newBoard, opponent_pieces_dict, not self.color)}")
            # print(f"fortify 2 = {fortify_analysis(newBoard, self.piecesDictionary, self.color)}")
            # build_opp_score[6] = (fortify_analysis(newBoard, opponent_pieces_dict, not self.color) - fortify_analysis(newBoard, self.piecesDictionary, self.color)) * self.fortifyWeight
            build_opp_score[6] = (fortify_analysis(newBoard, opponent_pieces_dict, not self.color)) * self.fortifyWeight

        if self.kingsPawnShieldWeight > 0.0:
            build_opp_score[7] = kings_pawn_shield(opponent_pieces_dict)

        if self.bishopPositionWeight > 0.0:
            build_opp_score[8] = bishop_positions(opponent_pieces_dict)

        if self.knightPositionWeight > 0.0:
            build_opp_score[9] = knight_positions(opponent_pieces_dict)

        if self.hideKingTuple[0] > 0.0 and self.hideKingTuple[1] >= turn_num:
            build_opp_score[2] = hideAnalysis(not self.color, opponent_pieces_dict["k"]) * self.hideKingTuple[0]

        if self.hideQueenTuple[0] > 0.0 and self.hideQueenTuple[1] >= turn_num:
            build_opp_score[3] = hideAnalysis(not self.color, opponent_pieces_dict["q"]) * self.hideQueenTuple[0]
            
        # Here, self.exp with be our explanation for the move we would have made from the opponent's position.
        # While, explain_move() will give our explanation for the enemy's move.
        # print(build_opp_score)
        retStr = ""
        build_opp_score = self.normalize(build_opp_score)
        if chosen_move.uci() == opp_move.uci():
            retStr += f"My opponent and I agree on {chosen_move.uci()}. " + self.exp
        else: 
            retStr += f"My opponent played {opp_move.uci()}, They must have been thinking this. Net score = {round(sum(build_opp_score), 2)}" + self.explain_move(turn_num, chosen_move, build_opp_score, opp_last_scoreList)
            retStr += f"\n\nI prefer {chosen_move.uci()}. " + self.exp
            

        return retStr

In [ ]:
# Instead of scoring each possible move during the selection period, the EngineAgent instead allows a chess engine to choose the next move to play.
# Then, it explains the chosen move using BuildableAgent's explanation process. 
class EngineAgent(BuildableAgent):
    def __init__(self, engine, color, norm_method, openingChoice, mobilityWeight, staticAnalysisWeight, safetyWeight, territoryWeight, fortifyWeight, hideKingTuple=(0,0), hideQueenTuple=(0,0)):
        """
        The buildable agent is the takes arguments that assign weights to its various scoring functions.
        When scoring or explaining moves, it uses all scoring methods that were not assigned a weight of zero.
        """
        self.engine = engine
        self.bestScore = -1
        self.exp = "no move yet made."
        self.color = color
        self.norm_method = norm_method
        self.opening = self.opening_select(openingChoice)
        self.init_pieces()
        self.mobilityWeight = mobilityWeight
        self.staticAnalysisWeight = staticAnalysisWeight
        self.safetyWeight = safetyWeight
        self.territoryWeight = territoryWeight
        self.fortifyWeight = fortifyWeight
        
        # hide tuples: weight, turns until the agent stops caring
        self.hideKingTuple = hideKingTuple
        self.hideQueenTuple = hideQueenTuple

        self.lastTurnScore = [0, 0, 0, 0, 0, 0, 0] # save the (normalized) scoreList of the previous turn.
    def move_select(self, color, analysing_opp, turn_num, board, own_pieces_dict, opponent_pieces_dict, last_scoreList):
        #first, score all moves same as the buildable agent does.
        #pick your favourite among them.
        #then, take stockfish's chosen move.
        #update piece dict based on stockfish's move.
        #generate explanation for stockfish's move
        #return stockfish's move.

        legal_moves = list(board.legal_moves)
        if not analysing_opp and self.opening:
            # then try to follow our saved opening plan. Pop the front entry in the list.
            next_opening_move = self.opening.pop(0)
            # print(f"popped {next_opening_move} from opening. Remaining: {self.opening}")
            
            if next_opening_move in [str(move.uci()) for move in legal_moves]:
                # then that move is valid. Do the following:
                # -update piece dict
                # -generate explanation
                # -return move
                start = next_opening_move[0:2]
                end = next_opening_move[2::]

                self.piecesDictionary = self.handle_castling(self.piecesDictionary, start, next_opening_move, color)
                opponent_pieces_dict = self.handle_en_passant(self.piecesDictionary, opponent_pieces_dict, start, end, next_opening_move, color, board)
                self.update_pieces(self.piecesDictionary, start, end)

                self.exp = "I'm following an opening and placing my trust in those who have come before me."

                # print("before returning my chosen move:")
                # print(f"self.piecesDictionary={self.piecesDictionary}")
                return chess.Move.from_uci(next_opening_move)
            else:
                self.opening.clear()

        movesScoresDict = {}
        for move in legal_moves:
          # print(f"considering move: {move.uci()}")
          # so we can break the scores down easier later, each score will be a list of scores, with each element corresponding to the score given by a certain scoring function. 

          # Legend (by index):
          # 0: mobility score
          # 1: static analysis score
          # 2: hide king score
          # 3: hide queen score
          # 4: safety score
          # 5: territory score
          # 6: fortify score
          movesScoresDict[move] = [0, 0, 0, 0, 0, 0, 0]

        #score each move
        movesScoresDict = self.score_move_dict(color, analysing_opp, turn_num, board, own_pieces_dict, opponent_pieces_dict, last_scoreList, movesScoresDict)
        
        # return move with highest score: (randomly choose between all the best)
        bestMoves = []
        self.bestScore = -1
        for move, scoreList in movesScoresDict.items():
            score = sum(scoreList)
            if self.bestScore == -1 or score > self.bestScore:
              bestMoves = [(move, scoreList)]
              self.bestScore = score
            elif score == self.bestScore:
              bestMoves.append((move, scoreList))

        own_chosen_pair = random.choice(bestMoves)
        own_chosen_move = own_chosen_pair[0]

        #however, stockfish prefers:
        sf_chosenMove = self.engine.play(board, chess.engine.Limit(time=0.1)).move
        #print(sf_chosenMove.move)

        #find the score for the move; it's already been calculated in movesScoresDict
        sf_chosenScoreList = movesScoresDict[sf_chosenMove]

        sf_chosen_scoreList = self.normalize(sf_chosenScoreList)
        
       
        # print(f"chosen_scoreList = {chosen_scoreList}")
        self.exp = f"Stockfish feels that this is the best move to proceed with.\nWe score it as {round(sum(sf_chosen_scoreList), 2)}." + self.explain_move(turn_num, sf_chosenMove, sf_chosen_scoreList, last_scoreList)

        if not analysing_opp: 
            # print(f"chosen_scoreList = {chosen_scoreList}")

            self.piecesDictionary = self.handle_castling(self.piecesDictionary, str(sf_chosenMove)[0:2], str(sf_chosenMove), color)
            opponent_pieces_dict = self.handle_en_passant(self.piecesDictionary, opponent_pieces_dict, str(sf_chosenMove)[0:2], str(sf_chosenMove)[2:4], str(sf_chosenMove), color, board)
            # print(f"1move chosen: opp dict after handling castle/en passant: {opponent_pieces_dict}")
            self.update_pieces(self.piecesDictionary, str(sf_chosenMove)[0:2], str(sf_chosenMove)[2:4])

            #print("before returning my chosen move:")
            #print(f"self.piecesDictionary={self.piecesDictionary}")

            self.lastTurnScore = sf_chosen_scoreList
       
        return sf_chosenMove

    """
    def interpret_opp_move(self, turn_num, board, opp_move, opponent_pieces_dict, opp_last_scoreList):
        # just say whether the engine agrees on the move choice or, if not, what move it prefers.
        pass
    """
        

In [ ]:
"""
Runs the game.
"""

if __name__ == "__main__":
    moves = 50
    
    #init legend:
    #(Buildable Agent)
    #__init__(self, color, norm_method, openingChoice, mobilityWeight, mobilityWeightList, staticAnalysisWeight, safetyWeight, territoryWeight, fortifyWeight, kingsPawnShieldWeights, bishopPositionWeights, knightPositionWeights, hideKingTuple=(0,0), hideQueenTuple=(0,0))    #(Engine Agent)
    #the same as buildable agent, just prepend stockfish to the arguments. 


    mobility_weights = {
        1 : 1, # pawn
        2 : 3, # knight
        3 : 3, # bishop
        4 : 5, # rook
        5 : 9, # queen
        6 : 0  # king
    }

    buildAgent1 = BuildableAgent(True, 0, "no opening", 1.0, mobility_weights, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, (0,0), (0,0))
    buildAgent2 = BuildableAgent(False, 0, "no opening", 1.0, mobility_weights, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, (0,0), (0,0))
    #buildAgent2 = BuildableAgent(False, 0, "adasdsdfe", 1.0, 5.0, 1.0, 1.0, 1.0)
    
    stockfish = chess.engine.SimpleEngine.popen_uci("stockfish-5-linux/Linux/stockfish_14053109_x64")
    
    #stockfish.configure({"Clear Hash": True})
    #engineAgent = EngineAgent(stockfish, True, 0, "psdwfe", 1.0, 1.0, 1.0, 1.0, 1.0, (0.0, 0), (0.0, 0))

    play_game(buildAgent1, buildAgent2, moves, "svg", 0.5)
    #play_game(engineAgent, buildAgent2, moves, "svg", 0.5)


Score breakdown explanation (Example):

•Score Type: previous turn's score -> current turn's score (weight applied to this score in the overall decision making process)


In [ ]:

def knight_positions(piecesDict):
    """
    returns 1 for each knight on one of the marked tiles.
    """
    score = 0
    markedTiles = ["c3", "c4", "c5", "c6", 
                   "d3", "d4", "d5", "d6",
                   "e3", "e4", "e5", "e6",
                   "f3", "f4", "f5", "f6"]

    for key, pos in piecesDict.items():
      #if the piece is in fact a knight, and if that knight is at markedTiles, add to score
      if key[0] == "n" and pos in markedTiles:
        score += 1

    return score

def bishop_positions(piecesDict):
    """
    returns 1 for each bishop on the longest diagonals.
    """
    score = 0
    markedTiles = ["a1", "b2", "c3", "d4", 
                   "e5", "f6", "g7", "h8",
                   "a8", "b7", "c6", "d5",
                   "e4", "f3", "g2", "h1"]

    for key, pos in piecesDict.items():
      #if the piece is in fact a knight, and if that knight is at markedTiles, add to score
      if key[0] == "b" and pos in markedTiles:
        score += 1

    return score

def kings_pawn_shield(piecesDict):
    """
    Returns points for every pawn adjacent to the king.
    """
    #for every piece
    # if the piece is a pawn
    #   and if the piece is adjacent to the king
    #     then add points
    
    def is_adj(pos1, pos2):
        """
        Returns true if the two positions entered are adjacent.
        The positions are format numberletter. e.g. "a2"
        """
        letterToX = {"a" : 1,"b" : 2,"c" : 3,"d" : 4,"e" : 5,"f" : 6,"g" : 7,"h" : 8}
        xToLetter = {1 : "a",2 : "b",3 : "c",4 : "d",5 : "e",6 : "f",7 : "g",8 : "h"}

        letter1 = letterToX[pos1[0]]
        letter2 = letterToX[pos2[0]]

        # two tiles are adjacent in the following conditions
        # -their letters differ by 1, and their numbers are the same
        # or
        # -their numbers differ by 1, and their letters are the same
        if abs(letter1 - letter2) == 1 and pos1[1] == pos2[1]:
            return True
        if abs(letter1 - letter2) == 0 and abs(int(pos1[1]) - int(pos2[1])) == 1:
            return True
        return False

    score = 0
    king_pos = piecesDict["k"]
    for key, pos in piecesDict.items():
        if key[0] == "p" and is_adj(king_pos, pos):
                score += 1

    return score

def territory_analysis(board):
    """
    perform the move under consideration on a copy board.
    dummyMove is used to trick the game into thinking it is your turn again.
    get all your own legal moves. (even though it isn't our turn again)
    create a set with the end coordinates of each move. e.g. (e3, g5, e4, etc...)

    return the length of the set
    """
    # switch sides so it's our player's turn again.
    board.push(chess.Move.null())

    our_moves = list(board.legal_moves)
    # print(f"our_moves = {our_moves}")
    territory_set = set()
    for move in our_moves:
        territory_set.add(move.uci()[2::])

    return len(territory_set)

def safety_analysis(board, piecesDict):
    """
    danger_pos_list is a string-list of all board spaces that the enemy can reach; i.e., spaces that are not safe.
    pieces dict is a dict (piece key: pos) that we want to check if vulnerable.

    return the number of pieces that are not vulnerable.
    """
    # -create a set with the end coordinates of each move. e.g. (e3, g5, e4, etc...)
    enemy_moves_now = list(board.legal_moves)
    danger_set = set()   
    for move in enemy_moves_now:
        danger_set.add(move.uci()[2::])

    counter = 0
    for key, pos in piecesDict.items():
        if pos not in danger_set:
            counter += 1

    # weight each piece by its material score?

    return counter

def hideAnalysis(color, pos):
    """
    if the pos is in the back two rows of the color in question, return 1.
    otherwise, return 0.
    """
    row = int(pos[1])
    if color:
        if row <= 2:
            return 1
    else:
        if row >= 7:
              return 1
    return 0

def staticAnalysis(board, my_color):
    """
    Returns an integer representing material score for a single player.
    """
    score = 0
    # Now check some other things:
    # **note. i think this is broken.

    for (piece, value) in [(chess.PAWN, 1), 
                          (chess.BISHOP, 4), 
                          (chess.KING, 0), 
                          (chess.QUEEN, 9), 
                          (chess.KNIGHT, 3), 
                          (chess.ROOK, 5)]:
        score += len(board.pieces(piece, my_color)) * value
    return score

def mobility_analysis(board, piece_weights, color):
    """
    Returns number of possible moves by the active player.
    Takes a board argument, and a pieces dictionary: {"piece_name" : piece_position, etc...}
    takes color argument to help pawns figure out which way to move.

    For each piece, calculate the number of moves it can make, then multiply that by its piece_score, which depends on the piece type.
    Return the sum of that as the positional analysis for the pieces given.

    (intrinsic weights removed so piece types can be weighted individually)
    piece_weights = {
        1 : 1, # pawn
        2 : 3, # knight
        3 : 3, # bishop
        4 : 5, # rook
        5 : 9, # queen
        6 : 0  # king
    }
    """
    # new version with our null move knowledge.
    switchedIt = False
    if board.turn is not color:
        switchedIt = True
        board.push(chess.Move.null())

    score = 0
    move_list = list(board.legal_moves)
    for move in move_list:
        piece_type = board.piece_at(chess.SQUARE_NAMES.index(move.uci()[0:2])).piece_type
        score += piece_weights[piece_type]

    if switchedIt is True:
        board.pop()
         
    return score

def fortify_analysis(board, piecesDict, color):
    """
    create empty set of pieces already fortified.
    for each piece:
      -check all moves. If it can move onto a friendly piece, add that piece to the set.
    return the len(set)
    """
    letterToX = {"a" : 1,"b" : 2,"c" : 3,"d" : 4,"e" : 5,"f" : 6,"g" : 7,"h" : 8}
    xToLetter = {1 : "a",2 : "b",3 : "c",4 : "d",5 : "e",6 : "f",7 : "g",8 : "h"}

    fortified_set = set()

    def isValidCoords(pos):
      # takes pos, a tuple (x, y) as an argument.
      # returns true if the position is within dimensions of a chess board. returns false otherwise.
      if pos[0] < 9 and pos[0] > 0 and pos[1] < 9 and pos[1] > 0:
          return True
      return False

    def allyOnPos(pos):
      # returns true if there is an ally piece on the pos.
      if board.piece_at(chess.SQUARE_NAMES.index(pos)) is not None and board.piece_at(chess.SQUARE_NAMES.index(pos)).color == color:
        return True
      return False

    def isEnemyCoords(pos):
      # returns true if there is an enemy piece on the pos.
      if board.piece_at(chess.SQUARE_NAMES.index(pos)) is not None and board.piece_at(chess.SQUARE_NAMES.index(pos)).color != color:
        return True
      return False

    def validate_pos(spot):
        # returns true means continue, no obstacles.
        # returns false means break out of loop, hit an obstacle/board edge.
        
        # if the spot exists on the board
        if isValidCoords(spot):
              # if there is an ally on the spot, add to fortified set and break.
              # else if there is an enemy on the spot, break
              spot_code = xToLetter[spot[0]] + str(spot[1])
              if allyOnPos(spot_code):
                # print(f"adding {spot_code}. Done by piece at {pos}")
                fortified_set.add(spot_code)
                return False
              elif isEnemyCoords(spot_code):
                return False
        else:
          return False
        return True


    def pawn(pos):
        # check the two possible diagonal spots, in the direction of colour.
        x = pos[0]
        y = pos[1]
        diagonalSpots = []
        if color:
            # if white at (5,5), then check (4,6) a (6,6)
            checkSpot = (x, y+1)
            if isValidCoords((x-1, y+1)):
              diagonalSpots.append((x-1, y+1))
            if isValidCoords((x+1, y+1)):
              diagonalSpots.append((x+1, y+1))
        else:
            # if black at (5,5), then check (4,4) a (6,4)
            checkSpot = (x, y-1)
            if isValidCoords((x-1, y-1)):
              diagonalSpots.append((x-1, y-1))     
            if isValidCoords((x+1, y-1)):
              diagonalSpots.append((x+1, y-1))
        for spot in diagonalSpots:
            # check if an ally is on that spot. If there is, add pos to set.
            spot_code = xToLetter[spot[0]] + str(spot[1])
            if allyOnPos(spot_code) is True:
                # print(f"adding {spot_code}. Done by piece at {pos}")
                fortified_set.add(spot_code)
       
    def knight(pos):
        # check the 8 possible spots.
        x = pos[0]
        y = pos[1]
        checkSpots = [(x-1, y+2), (x+1, y+2), (x-2, y+1), (x+2, y+1), (x-2, y-1), (x+2, y-1), (x-1, y-2), (x+1, y-2)]
        for spot in checkSpots:
            # if move is within the board AND move lands on an ally piece
            if isValidCoords(spot):
                spot_code = xToLetter[spot[0]] + str(spot[1])
                if allyOnPos(spot_code):
                    # print(f"adding {spot_code}. Done by piece at {pos}")                    
                    fortified_set.add(spot_code)

    def rook(pos):
        # explore in the rook's 4 legal directions. When exploring a direction:
        # - empty space? Proceed
        # - enemy on the space? break
        # - ally on the space? add to fortified_set and break
        x = pos[0]
        y = pos[1]

        # move to the left. Explore from (x, y) to (0, y)
        for i in range (x - 1, 0, -1):
          if not validate_pos((i, y)):
              break

        # move to the right. Explore from (x, y) to (8, y)
        for i in range (x + 1, 8+1):
          if not validate_pos((i, y)):
              break

        # move to the bottom. Explore from (x, y) to (x, 0)
        for i in range (y - 1, 0, -1):
          if not validate_pos((x, i)):
              break
        # move to the top. Explore from (x, y) to (x, 8)
        for i in range (y + 1, 8+1):
          if not validate_pos((x, i)):
              break
    
    def bishop(pos):
        # move to the top left. Explore from (x, y) to (0, 8)
        x = pos[0]
        y = pos[1]
        i = x
        j = y
        while i > 0 and j < 9:
          i -= 1
          j += 1
          spot = (i, j)
          if not validate_pos(spot):
              break

        # move to the bottom left. Explore from (x, y) to (0, 0)
        i = x
        j = y
        while i > 0 and j > 0:
          i -= 1
          j -= 1
          spot = (i, j)
          if not validate_pos(spot):
              break
        # move to the top right. Explore from (x, y) to (8,8)
        i = x
        j = y
        while i < 9 and j < 9:
          i += 1
          j += 1
          spot = (i, j)
          if not validate_pos(spot):
              break
        # move to the bottom right. Explore from (x, y) to (8,0)
        i = x
        j = y
        while i < 9 and j > 0:
          i += 1
          j -= 1
          spot = (i, j)
          if not validate_pos(spot):
              break
    
    def queen(pos):
        rook(pos)
        bishop(pos)
    
    def king(pos):
        return
    
    options = {
        1 : pawn,
        2 : knight,
        3 : bishop,
        4 : rook,
        5 : queen,
        6 : king
    }
    
    for key, pos in piecesDict.items():
        piece = board.piece_at(chess.SQUARE_NAMES.index(pos))
        piece_type = piece.piece_type
        options[piece_type]((letterToX[pos[0:1]], int(pos[1:])))

    # print(f"fortified_set = {fortified_set}")
    # print(fortified_set)
    return len(fortified_set)


Welcome to the genetic algorithm section, starting below:

In [ ]:
def assemble_piece_dict(board_position, colour):
  """
  Takes a board position and a player colour.
  Returns the piece dict for that player's colour.
  """
  p_count = 0
  kn_count = 0
  b_count = 0
  r_count = 0
  piece_dict = {}
  for i in range(0, 64):
    piece = board_position.piece_at(i)
    if piece is not None and colour == board_position.color_at(i):
      #print(piece)
      if piece.piece_type == chess.PAWN:
        piece_dict["p" + str(p_count)] = chess.SQUARE_NAMES[i]
        p_count += 1
      elif piece.piece_type == chess.KNIGHT:
        piece_dict["n" + str(kn_count)] = chess.SQUARE_NAMES[i]
        kn_count += 1
      elif piece.piece_type == chess.BISHOP:
        piece_dict["b" + str(b_count)] = chess.SQUARE_NAMES[i]
        b_count += 1
      elif piece.piece_type == chess.ROOK:
        piece_dict["r" + str(r_count)] = chess.SQUARE_NAMES[i]
        r_count += 1
      elif piece.piece_type == chess.QUEEN:
        piece_dict["q"] = chess.SQUARE_NAMES[i]
      else:  # (king)
        piece_dict["k"] = chess.SQUARE_NAMES[i]
  
  return piece_dict

In [ ]:
"""
Expert games dataset
We'll start with only 3 games, but should expand this later.
source for games: https://www.chess.com/games/search?p1=magnus-carlsen&sort=7

(25 games)
dataset format:
 -list of games
 -each game is a list of moves

"""
dataset = [
  ["d4", "Nf6", "c4", "e6", "Nf3", "d5", "Nc3", "c5", "cxd5", "cxd4", "Qxd4", "exd5", "Bg5", "Be7", "e3", "O-O", "Be2", "Nc6", "Qd3", "h6", "Bh4", "Be6", "O-O", "Qb6", "Bxf6", "Bxf6", "Nxd5", "Bxd5", "Qxd5", "Qxb2", "Bc4", "Qxa1", "Rxa1", "Bxa1", "g4", "Rae8", "h4", "Re7", "g5", "hxg5", "Nxg5", "Bf6", "Qf5", "Bxg5", "hxg5", "Re5", "Qf4", "Rc5", "g6", "Ne5", "gxf7+", "Nxf7", "Be6", "Rh5", "Qc7", "g5", "Qxb7", "Rh6", "Bb3", "g4", "Qxa7", "Kg7", "e4", "Rh5", "Qc7", "Kf6", "a4"],
  ["d4", "Nf6", "c4", "g6", "Nc3", "d5", "cxd5", "Nxd5", "e4", "Nxc3", "bxc3", "Bg7", "Bb5+", "c6", "Ba4", "b5", "Bb3", "a5", "Nf3", "O-O", "O-O", "a4", "Bc2", "c5", "Rb1", "Nc6", "e5", "Rb8", "Be4", "Bf5", "Bxf5", "gxf5", "Qd3", "Qd7", "Rd1", "Rfd8", "Bf4", "cxd4", "cxd4", "e6", "Bg5", "Rdc8", "Qe3", "Ne7", "Bf6", "h6", "Qf4", "Ng6", "Qg3", "Kh7", "d5", "exd5", "Rb4", "Rc4", "Rxc4", "bxc4", "e6", "Qxe6", "Qxb8", "Bxf6", "Re1", "Qd7", "Qb6", "Kg7", "g3", "f4", "Qc5", "fxg3", "hxg3", "Nf4", "Qe3", "Nd3", "Rb1", "Qe6", "Rb7", "Qxe3", "fxe3", "c3", "Rc7", "Nb4", "Ne1", "Be5", "Rc8", "Bxg3", "Kf1", "Bxe1", "Kxe1", "Nxa2", "Kd1", "a3", "Rb8", "h5", "Kc2", "h4", "Rb1", "Kf6", "Kb3", "Nb4", "Kxb4", "a2", "Ra1", "c2", "Kc3", "h3", "Kxc2", "Kg5", "Kd3"],
  ["Nf3", "d5", "g3", "g6", "Bg2", "Bg7", "d4", "Nf6", "O-O", "O-O", "c4", "c6", "Nc3", "dxc4", "e4", "Bg4", "h3", "Bxf3", "Bxf3", "e5", "dxe5", "Nfd7", "e6", "fxe6", "Be3", "Qe7", "Bg2", "Na6", "Qe2", "Nb4", "Qxc4", "Nc2", "Nd5", "cxd5", "Qxc2", "d4", "Bd2", "Rac8", "Qb3", "Nc5", "Qa3", "Qd7", "Rac1", "b6", "e5", "Bxe5", "Rfe1", "Bg7", "b4", "Na4", "Rxc8", "Rxc8", "Qb3", "Nc3", "Rxe6", "Kh8", "Kh2", "Re8", "Rxe8+", "Qxe8", "Qc4", "h6", "a3", "a5", "bxa5", "bxa5", "Bf1", "Qf8", "Kg2", "Ne4", "Be1", "Qxa3", "Bd3", "Nd6", "Qa6", "Nf5", "h4", "h5", "Bxa5", "Kh7", "Qb5", "Qf8", "Qd5", "Qd6", "Qf3", "Qe5", "Bd8", "Qe6", "Qb7", "Nd6", "Qc6", "Qb3", "Bxg6+", "Kxg6", "Qxd6+", "Kh7", "Qd7", "Kg6", "Qc6+", "Kf7", "Qc5", "Qb7+", "f3", "Ke8", "Bc7", "Qb2+", "Kh3", "d3", "Qxh5+", "Kd7", "Qf7+", "Kc6"],
  ["e4","e5","Nf3","Nc6","d4","exd4","Nxd4","Bb4+","c3","Be7","Bf4","Nf6","e5","Nd5","Bg3","O-O","Nf5","d6","Nxe7+","Ndxe7","exd6","Nf5","Be2","Re8","O-O","Nxg3","hxg3","Qxd6","Qxd6","cxd6","Re1","d5","Nd2","d4","Bf3","Bd7","c4","Ne5","Rad1","Bc6","Bxc6","Nxc6","Rxe8+","Rxe8","Kf1","f5","Nf3","Kf7","a3","a5","b4","axb4","axb4","Nxb4","Rb1","Nc6","Rxb7+","Re7","Rb5","Kf6","Rd5","Re4","c5","g6","Kg1","Ke7","Kh2","Kf6","Kh3","h6","Kh2","Ne5","Rd6+","Kf7","Nxe5+","Rxe5","Rxd4","Rxc5","g4","fxg4","Rxg4"],
  ["d4","Nf6","c4","g6","Nc3","Bg7","e4","d6","Nge2","O-O","Ng3","e5","d5","a5","Be2","Na6","h4","h5","Bg5","Qe8","Qd2","Nc5","O-O-O","Ng4","Bxg4","Bxg4","f3","Bd7","Be3","b6","Kb1","Kh7","Qc2","a4","Nge2","f5","exf5","gxf5","Rh3","Kh8","f4","Ne4","Nxe4","fxe4","Rg3","Bg4","Rxg4","hxg4","f5","Rxf5","Ng3","Rf8","Qxe4","Qd7","a3","b5","c5","dxc5","h5","c4","h6","Bf6","Bc5","Rf7","Rf1","Re8","Bb4","Bg5","Nf5","c6","Bd6","Bf4","Ng7","Qxd6","Nxe8","Qxd5","Qxd5","cxd5","g3","Kh7","gxf4","exf4","Nd6","Rf6","Nxb5","f3","Nd4","Kxh6","Kc2","Kg5","Kd2","f2","Ne2","Rf3","Kc2","Kh4","Rh1+","Rh3","Rf1","g3","Kd2","Kg4"],
  
  ["d4","Nf6","c4","e6","Nf3","b6","g3","Ba6","Nbd2","Bb7","Bg2","Be7","O-O","O-O","b3","c5","Bb2","cxd4","Nxd4","Bxg2","Kxg2","Nc6","Nxc6","dxc6","e4","Qd3","e5","Ne4","Nf3","Rfd8","Re1","Nc5","Nd4","Rac8","Re3","Qxd1","Rxd1","Nxb3","axb3","c5","Red3","cxd4","Rxd4","Kf8","Kf3","Ke8","Ke4","Bc5","Rxd8+","Rxd8","Rxd8+","Kxd8","f3","Kc7","Bc3","Kc6","Kd3","b5","b4","Bg1","h3","Bf2","g4","Bg3","Kd4","a6","Kd3","Bf2","Bd2","Bg3","Bc3","Bf2","Bd2","Bg3","Bc3"],
  ["d4","Nf6","Nf3","d5","c4","e6","Nc3","Bb4","cxd5","exd5","Bg5","h6","Bh4","g5","Bg3","Ne4","Nd2","Nxc3","bxc3","Bxc3","Rc1","Bb2","Rxc7","Na6","Rc2","Bxd4","e3","Bg7","h4","Nb4","Rc7","O-O","hxg5","Qxg5","Bd6","Nc6","Bxf8","Kxf8","Qf3","Be6","Rxb7","Rc8","Qf4","Qxf4","exf4","Nd4","Nb3","Nxb3","axb3","Rc2","Kd1","Rxf2","Be2","Rxf4","Rxa7","Rd4+","Kc1","Rb4","Bh5","Bd4","Ra4","Be3+","Kb2","Rb6","Be2","Bd7","Ra5","Be6","Rh5","Kg7","Raxd5","Bxd5","Rxd5","Bf4","Bf3","Bd6","Kc3","Bb4+","Kc4","Be7","Rd7","Kf6","Bd5","Rb4+","Kc3","Rb6","Rb7","Rxb7"],
  ["d4","Nf6","c4","g6","Nc3","d5","Nf3","Bg7","Bg5","Ne4","Bf4","Nxc3","bxc3","c5","cxd5","cxd4","cxd4","Qxd5","e3","O-O","Be2","Nc6","O-O","Bf5","Qa4","Qa5","Qxa5","Nxa5","Rfc1","Rfc8","Nd2","Rxc1+","Rxc1","Rc8","Rxc8+","Bxc8","Bf3","Nc6","Bc7","Kf8","Nb3","Ke8","Nc5","e6","Bxc6+","bxc6","Bb8","a6","Bd6","Bf6","Kf1","Bd8","Ke2","Bb6","Na4","Bd8","Nc5","Bb6","Ne4","Kd7","Be5","Bd8","Bf6","Bb6","Ng5","c5","Nxh7","Bb7","g4","Bd5","a3","Bc4+","Kd1","Bd5","Ng5","cxd4","Bxd4","Bxd4","exd4","f6","Nh7","Ke7","g5","fxg5","Kd2","e5","Ke3","Ke6","Nf8+","Kf5","dxe5","Kxe5","Nxg6+","Kf6","Kd4","Bg2","Ne5","Kf5","Nd3","Kg4","Ke3","Kh3","Nc5","a5","f3","g4","fxg4","Kxg4","Nb3","a4","Nc5","Bc6","Kf2","Kh3","Kg1","Kg4","Nd3","Bb5","Nb4","Kh3","Na2","Be8","Nc3","Kg4","Nd1","Bb5","Nf2+","Kf3","h4","Be8","Nd1","Kg3","Nc3","Kxh4","Kf2","Kg4","Ke3","Kf5","Kd4","Ke6","Kc5","Ke7","Nxa4","Bxa4","Kb4","Be8","a4","Bxa4"],
  ["d4","Nf6","c4","g6","f3","d5","cxd5","Nxd5","e4","Nb6","Nc3","Bg7","Be3","O-O","Qd2","Nc6","O-O-O","Qd6","Nb5","Qd7","Kb1","Rd8","d5","a6","Nc3","Qe8","Qc1","Na5","Bh6","Bxh6","Qxh6","e6","Nh3","Qe7","Bd3","e5","Nf2","Nbc4","h4","Rd6","Bxc4","Nxc4","Qc1","b5","Nd3","Bd7","b3","Nb6","h5","g5","g3","c6","f4","cxd5","Nxe5","d4","Qa3","a5","Nxb5","Bxb5","Rxd4","Re6","Qxe7","Rxe7","Rc1","Nd7","Rc7","Nxe5","Rxe7","Nc6","Rd5","Bd3+","Rxd3","Nxe7","fxg5","Rb8","Rd7","Kf8","Ra7","Rb5","Ra8+","Kg7","Re8","Re5","g4","Rxe4","Kc2","Re5","Kd3","f6","gxf6+","Kxf6","Rh8","Kg7","Re8","Kh6","a3","Kg5","Rh8","h6","Rh7","Re6","Rg7+","Kf6","Rh7","Ke5","Rg7","Kf4","b4","axb4","axb4","Nc6","b5","Ne5+","Kd4","Nxg4","Kc5","Re5+","Kc6","Rxh5","b6","Ne5+","Kc7","Nc4","b7","Rc5+","Kd8","Rb5","Kc7","Rc5+","Kd8","Rb5","Kc7","Ke5"],
  ["c4","c5","Nf3","Nf6","Nc3","d5","cxd5","Nxd5","e3","Nxc3","bxc3","Qc7","d4","g6","Bb5+","Bd7","a4","Bg7","O-O","O-O","Ba3","b6","dxc5","bxc5","Qd5","Bxb5","axb5","Nd7","Rfd1","Rfd8","Ng5","e6","Qc6","Qxc6","bxc6","Ne5","c7","Rdc8","f4","h6","Ne4","Nc4","Rd7","Nb6","Rad1","Nxd7","Rxd7","Bf8","c4","a5","Nc3","a4","Nb5","Re8","e4","Rac8","Na7","Ra8","Nb5","Rac8","Na7","Ra8","Nb5"],
  
  ["d4","d5","c4","c6","cxd5","cxd5","Nc3","Nf6","Bf4","Nc6","e3","Bf5","Rc1","Rc8","Nf3","e6","Qb3","Bb4","Bb5","Bxc3+","bxc3","O-O","Bxc6","Rxc6","Qxb7","Qc8","Qxc8","Rfxc8","Ne5","Rxc3","Rxc3","Rxc3","O-O","h6","h4","Ne4","g4","Bh7","Rb1","g5","hxg5","hxg5","Bh2","Nd2","Rb8+","Kg7","Rb7","Be4","Rxf7+","Kg8","f3","Nxf3+","Rxf3","Bxf3","Nxf3","Rc1+","Kf2","Rc2+","Ke1","Rxa2","Bd6","a5","Nxg5","a4","Kd1","Rb2","Nxe6","Rb6","Nf4","Rxd6","Kc2","Rb6","Nxd5","Rb7","Nc3","a3","e4","Kf7","e5","Ke6","Kc1","Rc7","Kd2","Ra7","Na2","Rb7","Kc3","Rb8","g5","Kf5","d5","Kxe5","g6","Kxd5","g7","Ke6","g8=Q+","Rxg8","Kb3","Rg3+","Kb4","Ke5","Nc3","Re3","Kc4","Kf5","Kb4","Kg4","Na2","Kf3","Nc3","Kg2","Nd5","Rf3","Nc3","Kf1","Kc4","Ke1","Kb4","Rh3"],
  ["d4","d5","c4","c6","Nf3","Nf6","Nbd2","Bf5","Nh4","Be4","Qb3","Qb6","e3","e6","Be2","h6","Nxe4","dxe4","g3","c5","Bd2","Nc6","Bc3","cxd4","exd4","O-O-O","O-O-O","Bb4","Ng2","Bxc3","bxc3","e5","Qxb6","axb6","d5","Nb8","Ne3","Na6","g4","Nc5","h4","Ne8","h5","Nd6","Rhg1","f6","Nf5","Nxf5","gxf5","Rd7","Rg4","Rf8","Rdg1","Rff7","Kd2","Kc7","Ke3","Na4","Bd1","Nc5","Bc2","Kd8","Bxe4","Ke8","Bc2","Kf8","Rb1","Rd6","Rb5","Rc7","Rg1","Nd7","Rgb1","Rxc4","R1b3","Rh4","Bd1","Rf4","R3b4","Rxb4","cxb4","Ke7","a4","Kd8","a5","Kc7","Ba4","Kd8","Kd3","Kc7","Bb3","Kd8","Kc3","Kc7","f3","bxa5","Rxa5","Nb6","Rc5+","Kd8","Kb2","Nc8","Ra5","Kc7","Ra1","Rd7","Rg1","Nd6","Bc2","Kb6","Bd3","Rc7","Kb3","Re7","Kc3","Rc7+","Kd2","Rd7","Ke3","Re7","Rg2","Rc7","Kd2","Re7","Ke3","Rc7","Kd2","Re7"],
  ["e4","e5","Nf3","Nc6","Bb5","Nf6","d3","Bc5","Nbd2","Nd4","Nxd4","Bxd4","c3","Bb6","O-O","c6","Ba4","O-O","Bb3","d5","h3","dxe4","dxe4","Qe7","Qf3","Be6","Nc4","Nd7","Ne3","Bxe3","Bxe3","Rfd8","Rfd1","a6","Bxe6","Qxe6","Qg4","Qxg4","hxg4","h6","a4","Kf8","a5","Ke8","f3","Nf8","Bb6","Rxd1+","Rxd1","Nd7","Be3","Rd8","Kf2","f6","Ke2","Nf8","Rh1","Ne6","g3","Nf8","f4","Ng6","g5","exf4","gxf4","fxg5","fxg5","hxg5","Bxg5","Rd7","Rg1","Kf7","Be3","Nf8","Bd4","Ne6","Ke3","Rd8","Be5","Rh8","Rg3","Rh1","Rf3+","Ke7","b4","Re1+","Kd3","Rd1+","Ke2","Rh1","Ke3","Re1+","Kd3","Rd1+","Ke2","Rh1"],
  ["e4","e5","Nf3","Nc6","d4","exd4","Nxd4","Bc5","Nb3","Bb6","Nc3","Nf6","Bg5","O-O","Qf3","Nd4","Nxd4","Bxd4","O-O-O","Bxc3","bxc3","Re8","Bc4","d6","Bxf6","Qxf6","Qxf6","gxf6","Rhe1","f5","Bd5","fxe4","Bxe4","Re5","Bd5","Rxe1","Rxe1","Kf8","Re4","c6","Bb3","Bf5","Rf4","Bg6","h4","h5","Kd2","Rd8","Bc4","d5","Bd3","Kg7","a4","b6","g3","c5","Be2","Rd6","a5","Rf6","axb6","axb6","Bf3","Rd6","Bg2","Rd8","Bf3","Rd6","Bg2","Rd8","Ra4","Rd6","Rf4","b5","Bf1","Rb6","Bg2","Ra6","Bf1","Ra2","Bd3","Bxd3","Kxd3","Kg6","g4","hxg4","Rxg4+","Kf5","Rg5+","Ke6","Rg1","Ra3","Kd2","b4","cxb4","cxb4","Rg3","Ra1","Rh3","Kf6","h5","Kg7","h6+","Kh7","Rh4","Ra6","Rxb4","Kxh6","Rb5","Rf6","Ke2","Re6+","Kd3","Rf6","Rxd5","Rxf2","c4","Rf1","c5","Rc1","Kd2","Rc4","Kd3"],
  ["d4","Nf6","c4","c5","Nf3","cxd4","Nxd4","a6","Nc3","e6","e4","Qc7","a3","b6","Be3","Bb7","f3","Nc6","Rc1","Nxd4","Bxd4","Bd6","g3","h5","Qd2","h4","g4","Bf4","Be3","Bxe3","Qxe3","Qc5","Kf2","Bc6","Be2","a5","e5","Ng8","b4","axb4","axb4","Qxb4","Ra1","Ne7","Rxa8+","Bxa8","Rb1","Qa5","Qxb6","Qxb6+","Rxb6","O-O","f4","g5","fxg5","Ng6","Rb5","Bc6","Ra5","Rb8","Nb5","Nxe5","Nd4","Ng6","Ke3","Rb2","Rc5","Ne7","Ra5","Kg7","Ra7","Kg6","Nxc6","Nxc6","Rxd7","Ne5","Rd2","Rxd2","Kxd2","Kxg5","Ke3","Nd7","Bf3","f5","gxf5","Kxf5","Bc6","Ne5","Be4+","Kf6","c5","Ng4+","Kf4","Nxh2","c6","Ke7","Kg5","Kd6","Kxh4","Nf1","Kg5","Nd2","Bg2","Ke5"],

  ["e4","e5","Nf3","Nc6","Bc4","Bc5","O-O","Nf6","d3","O-O","Re1","Ng4","Re2","Kh8","h3","f5","Bg5","Nf6","Nc3","d6","Nd5","fxe4","dxe4","Be6","Nxf6","gxf6","Bxe6","fxg5","c3","Qf6","Bg4","Ne7","Qd2","Rg8","b4","Bb6","a4","a5","Qa2","Qg6","Qe6","Qg7","bxa5","Bxa5","Rb1","Ra7","Qc4","Qg6","Rb5","c6","Nxe5","dxe5","Rxe5","Rd8","Re6","Qg7","g3","Raa8","Kg2","Ng6","e5","Bc7","Bf5","Nxe5","Qb4","Rf8","Re7","Rf7","Qxb7","Raf8","Rxf7","Qxf7","Be4","h5","Bxc6","Qc4","Bb5","Qxc3","Qe4","Qc5","Rc2","Qd6","Be2","h4","gxh4","Ng6","Kf1","Bb6","Bf3","Ne5","Rd2","Qc5","Rc2","Nxf3","Rxc5","Nd2+","Ke2","Nxe4","Rc6","Bxf2","hxg5","Nxg5","a5","Ne4","Kd3","Nc5+","Kc4","Nb7","Rh6+","Kg7","Rh5","Ra8","Rf5","Nd6+"],
  ["d4","Nf6","Nf3","d5","c4","e6","Nc3","dxc4","e3","a6","a4","b6","Bxc4","Bb7","O-O","Bb4","Qe2","O-O","Rd1","Qe7","Bd3","c5","Na2","Nc6","Nxb4","Nxb4","b3","Nxd3","Rxd3","Be4","Rd1","Rfc8","Ba3","Qb7","dxc5","bxc5","Rdc1","Qxb3","Bxc5","Nd7","Bd4","Rxc1+","Rxc1","Qxa4","Ne5","Nxe5","Bxe5","Bd5","f3","f6","Bd6","Rd8","Qb2","Bb3","Qa3","Qb5","h3","e5","Rb1","Qd3","Qxb3+","Qxb3","Rxb3","Rxd6","Rb7","h5","h4","Rd3","e4","a5","Ra7","Ra3","Kf2","a4","Ra8+","Kf7","Ra7+","Kg6","Kg3","Ra2","Kh3","Ra1","g3","Ra2","f4","a3","fxe5","fxe5","g4","Ra1","g5","a2","Kg2","Kh7","Kh2","Kg6","Kg2","Re1","Ra6+","Kf7","Rxa2","Rxe4","Kh3","Rd4","Ra7+","Kg6","Re7","Re4","Kg3","Rg4+","Kh3","e4","Ra7","Rf4","Re7","Rg4","Ra7","Kf5","Rxg7","e3","Rf7+","Ke4","Re7+","Kd3","Rd7+","Kc2","Re7","Kd2","Rd7+","Ke1","Rh7","Kf2","Rf7+","Ke1","Rh7","e2","Rxh5","Re4","g6","Kd2","Rd5+","Ke3"],
  ["e4","e5","Nf3","Nc6","Bb5","a6","Ba4","Nf6","O-O","Bc5","c3","b5","Bb3","d6","a4","Bb7","d3","h6","Nh4","Rb8","axb5","axb5","Nd2","O-O","Ndf3","Ne7","h3","Bb6","Nh2","Kh8","Ng4","Nfg8","Be3","c5","Bc2","Nd5","Nf5","Nxe3","Nfxe3","g6","c4","h5","Nh2","bxc4","Nxc4","Bc7","Nf3","Kg7","Ne3","Ne7","Bb3","Bc8","Nd2","Nc6","Bd5","Nd4","Ra2","Ne6","Bxe6","Bxe6","Ndc4","Qg5","Ra7","Rfc8","Qf3","Rb3","Nd5","Bb8","Ra3","Rb7","Rfa1","Rf8","Ra6","Rd7","Rb6","f5","Qg3","Qxg3","fxg3","fxe4","dxe4","Bxd5","exd5","Bc7","Rb7","Re7","Ra3","e4","Re3","Rf5","Nd2","Rxd5","Rxe4","Rf7","Nf3","Rd1+","Re1","Rxe1+","Nxe1","d5","Rb5","Bxg3","Nf3","Ra7","Kf1","Ra1+","Ke2","Rc1","b4","cxb4","Rxb4","Rc2+","Kd3","Rxg2","Rd4","Bb8","h4","Rf2","Ke3","Ra2","Rxd5","Ba7+","Kf4","Bf2","Rd7+","Kh6","Re7","Ra4+","Re4","Ra5","Re6","Ra4+","Re4","Bg3+","Ke3","Ra3+","Ke2","Ra2+","Ke3","Kg7","Rb4","Ra3+","Ke4","Ra7","Ke3","Ra3+","Ke4","Ra6","Ke3","Re6+","Re4","Rxe4+","Kxe4","Kf6","Ke3","Kf5","Kd2","Kg4","Ke2","Kh3","Ng5+","Kxh4","Ne6","Bd6","Kf3","Kh3","Ng5+","Kh2","Ne6","Be5","Ke4","Bb8","Kf3","Bd6","Kf2","h4","Ng5","Bc5+","Kf3","Be7","Ne4","h3","Nf2","Bh4"],
  ["b3","d5","Bb2","Bf5","Nf3","e6","g3","Nf6","Bg2","Be7","O-O","O-O","c4","c6","d3","h6","Nbd2","a5","a3","Nbd7","Ra2","Bh7","Qa1","Re8","Rc1","Bd6","Ne5","Nxe5","Bxe5","Bf8","Bc3","Nd7","b4","Nb6","bxa5","Na4","Bb4","c5","Nb3","b6","Be1","bxa5","Qe5","Qb6","Rb1","Rab8","Raa1","Bd6","Qh5","Bg6","Qf3","Be5","Nd2","Qc7","Rxb8","Rxb8","Rc1","Bb2","Rc2","d4","Nb1","Ba1","Rc1","Bb2","Rc2","Nc3","Nxc3","Bxc3","Rc1","Bb2","Rb1","Bxa3","Rxb8+","Qxb8","Bd2","Bb4","Bf4","Qa7","Qc6","a4","Qc8+","Kh7","Bb8","Qe7","Qa6","a3","Be5","f6","Bf4","e5","Bc1","e4","dxe4","Bxe4","Bf1"],
  ["d4","Nf6","c4","e6","Nf3","d5","g3","Bb4+","Bd2","Be7","Bg2","O-O","O-O","Nbd7","Qb3","Nb6","c5","Nc4","Bc3","b6","c6","a5","Nbd2","Ba6","Nxc4","Bxc4","Qc2","a4","Ne5","Bb5","Rfe1","Bd6","Bd2","a3","b3","Ne4","Bxe4","dxe4","Nd7","f5","Nxf8","Qxf8","Bf4","Rd8","Be5","Bxe5","dxe5","h6","Rad1","Rxd1","Rxd1","Qb4","Kf1","e3","f3","h5","Rd7","h4","Rxc7","h3","Rd7"],

  ["d4","Nf6","c4","e6","Bf4","d5","cxd5","Nxd5","Bg3","c5","e4","Nf6","Nd2","Nc6","e5","Nd7","Ngf3","cxd4","Bc4","Be7","O-O","O-O","Rc1","Nc5","a3","a5","Nb3","b6","Nbxd4","Nxd4","Nxd4","Bb7","Nb5","Ne4","Nd6","Qb8","Qd4","b5","Bxb5","Nxg3","hxg3","Bd5","Bc6","Bxd6","exd6","Ra6","Bxd5","Rxd6","Rc5","Rxd5","Rxd5","exd5","Qxd5","Qxb2","Qxa5","g6","Qb4","Qf6","a4","Rc8","a5","h5","Qa4","Rc6","Qa1","Kg7","Rb1","Qxa1","Rxa1","Ra6","f3","Kf6","Kf2","Ke5","Ke3","Kd5","Kf4","f6","g4","hxg4","Kxg4","Ke5","f4+","Ke6","Ra2","Ke7","Kf3","Kd6","Ke4","Ke6","Ra1","Kd6","f5","gxf5+","Kxf5","Ke7","Kg6","Ke6","Re1+","Kd7","g4","Kd8","Kf7","Rd6","Re4","Rc6","Rd4+","Kc8","Rf4"],
  ["e4","e5","Nf3","Nc6","Bb5","a6","Ba4","Nf6","O-O","Be7","Re1","b5","Bb3","d6","c3","O-O","a3","h6","d4","Re8","Nbd2","Bf8","Bc2","Bg4","h3","Bh5","d5","Nb8","a4","Nbd7","Qe2","Rb8","Nb3","c6","dxc6","Nc5","axb5","axb5","Nxc5","dxc5","g4","Bg6","c4","bxc4","Bd2","Qc8","Bc3","Bd6","Nh4","Kh7","Qxc4","Re7","Red1","Qxc6","Nxg6","fxg6","Ra6","Rb6","Ba4","Rxa6","Bxc6","Rxc6","b4","Rb7","b5","Rcb6","Kg2","Bb8","Rb1","Ne8","Qxc5","Nd6","Qxe5","Nxb5","Qc5","Bd6","Qe3","g5","e5","Nxc3","Rxb6","Rxb6","Qd3+","Kh8","exd6"],
  ["e4","c6","d4","d5","e5","c5","dxc5","e6","Qg4","Nd7","Nf3","Nxc5","Be2","Ne7","O-O","Nf5","c4","dxc4","Rd1","Bd7","Bxc4","Qb6","Nc3","Be7","Nd4","h5","Qe2","Nxd4","Rxd4","Bc6","Be3","a5","Rad1","O-O","Qxh5","Qxb2","Rg4","g6","Bh6","Be8","Nd5","exd5","Bxd5","Kh7","Qh3","Rh8","Bf8+","Kg8","Rxg6+","Kxf8","Qxh8#"],
  ["d4","Nf6","c4","g6","Nc3","d5","h4","c5","cxd5","Nxd5","Na4","Nc6","e4","Nb6","d5","Ne5","h5","Nxa4","Qxa4+","Bd7","Qa3","Qb6","Qc3","Qb4","Qxb4","cxb4","f4","Ng4","hxg6","fxg6","e5","Bf5","Be2","Bg7","Bf3","O-O","Ne2","h5","Nd4","Bd7","Bd2","Rac8","Rc1","Rxc1+","Bxc1","Rc8","Ke2","Rc4","Kd3","Rc5","Be3","Bh6","g3","Nxe3","Kxe3","a5","Be4","Kg7","Nb3","Rc8","Nxa5","Ra8","Nxb7","Rxa2","Nc5","Bg4","Nd3","g5","f5","h4","gxh4","gxh4+","Kd4"],
  ["Nf3","d5","g3","g6","Bg2","Bg7","O-O","Nf6","c4","c6","b3","Ne4","d4","O-O","Bb2","Bf5","e3","a5","Nc3","Nd7","Qe2","Nxc3","Bxc3","Be4","cxd5","cxd5","Qb5","Qc7","Rfc1","Qc6","Qxc6","bxc6","Ne1","g5","Bxe4","dxe4","Rc2","Rfb8","Rac1","a4","b4","a3","Bd2","e6","Rxc6","Bf8","Nc2","Nf6","Rb1","Nd5","Rb3","g4","b5","Rb7","Bc1","Rba7","Rc5","Ra4","Bxa3","Bxc5","dxc5","e5","Nb4","Nxb4","Bxb4","Rxa2","b6","Rc2","Bc3","f6","b7"],
]

def assemble_fitness_set(number_of_moves):
  """
  Used to create the testing set for the agent to have its fitness scored against.
  Assembled randomly.
  Returns list of (board, next move in san, list of position dicts (parallel to list of legal moves))
  """
  fitness_set = []

  for i in range(0, number_of_moves):
    # move-score process:
    # pick a game, pick some depth into that game.
    game = random.choice(dataset)
    position = random.randint(4, len(game) - 3)  # start at min 5 deep into the game, since at the start, a great many moves are all valid.

    # advance a board to that point in the game.
    board = chess.Board()
    for j in range(0, position):
      board.push_san(game[j])
     
    # compare to real next move. (which is in san format)
    real_next_move = game[position]

    # create list of position dicts for all legal moves
    legal_moves = list(board.legal_moves)
    position_dicts_list = []
    for lm in legal_moves:
      dummy_board = board.copy()
      dummy_board.push(lm)
      piece_dict = assemble_piece_dict(dummy_board, not board.turn)
      position_dicts_list.append(piece_dict)
      
    fitness_set.append((board, real_next_move, position_dicts_list))

  return fitness_set

import chess
import chess.engine
import chess.svg

# test that all the games can be played through properly.
for game in dataset:
  board = chess.Board()
  for i in range(0, len(game)):
    board.push_san(game[i])

In [ ]:
def generate_weight(index):
  """
  Returns a value in the firstgen agent range for the corresponding weight. 
  Ranges:
   -mobility: 0 to 10
   -mobility list: [ dummy, pawn, knight, bishop, rook, queen, king ]; 0 to 10 for all of them.
   -static: 0 to 10
   -safety: 0 to 10
   -territory: 0 to 10
   -fortify: 0 to 10
   -kings pawn shield: 0 to 10
   -bishop position: 0 to 10
   -knight position: 0 to 10
   -hide king: (0 to 20, 0 to 10)
   -hide queen: (0 to 20, 0 to 10)
  """
  if index == 0:
    return random.randint(0, 10)
  elif index == 1:
    mob_weight_list = {
        1 : random.randint(0, 10), # pawn
        2 : random.randint(0, 10), # knight
        3 : random.randint(0, 10), # bishop
        4 : random.randint(0, 10), # rook
        5 : random.randint(0, 10), # queen
        6 : 0  # king
    }
    return mob_weight_list
  elif index == 2:
    return random.randint(0, 10)
  elif index == 3:
    return random.randint(0, 10)
  elif index == 4:
    return random.randint(0, 10)
  elif index == 5:
    return random.randint(0, 10)
  elif index == 6:
    return random.randint(0, 10)
  elif index == 7:
    return random.randint(0, 10)
  elif index == 8:
    return random.randint(0, 10)
  elif index == 9:
    return (random.randint(0, 20), random.randint(0, 10))
  elif index == 10:
    return (random.randint(0, 20), random.randint(0, 10))

def create_firstgen_agent():
  """
  Returns a weightlist of the following format:
  [mobility, mobility list, static, safety, territory, fortify, kings pawn shield, bishop position, knight positions, hide king, hide queen]
  """
  return [ generate_weight(0), generate_weight(1), generate_weight(2), generate_weight(3), generate_weight(4), generate_weight(5), generate_weight(6), generate_weight(7), generate_weight(8), generate_weight(9), generate_weight(10) ]

In [ ]:
def individual_select_move(board, colour, ind, piece_dict_list):
  """
  Given some board state, and some individual
  return the best move in the eyes of that individual.
  """
  normals = [100, 1, 41, 16, 16, 16, 4, 2, 2, 1, 1 ]
  moves = list(board.legal_moves)

  best_move = None
  best_score = -1
  for i in range(0, len(moves)):#move in moves:
    nb = board.copy()
    nb.push(moves[i])

    score = 0
    piece_dict = piece_dict_list[i]
    # piece_dict = assemble_piece_dict(nb, colour)
    turn_num = nb.fullmove_number

    score += (mobility_analysis(nb, ind[1], colour) * ind[0] / normals[0])
    score += (staticAnalysis(nb, colour) * ind[2] / normals[2])
    score += (safety_analysis(nb, piece_dict) * ind[3] / normals[3])
    score += (territory_analysis(nb) * ind[4] / normals[4])
    score += (fortify_analysis(nb, piece_dict, colour) * ind[5] / normals[5])
    score += (kings_pawn_shield(piece_dict) * ind[6] / normals[6])
    score += (bishop_positions(piece_dict) * ind[7] / normals[7])
    score += (knight_positions(piece_dict) * ind[8] / normals[8])
    if "k" in piece_dict and ind[9][0] < turn_num:
      score += (hideAnalysis(colour, piece_dict["k"]) * ind[9][1] / normals[9])
    if "q" in piece_dict and ind[10][0] < turn_num:
      score += (hideAnalysis(colour, piece_dict["q"]) * ind[10][1] / normals[10])

    score = round(score, 2)
    if best_score == -1 or score > best_score:
      best_score = score
      best_move = moves[i]

  # return the highest scoring move (or a random one if the highest-scoring moves are tied)
  # returns in san format
  return board.san(best_move)


# Testing
"""
b = chess.Board()
weight_list = create_firstgen_agent()
result = individual_select_move(b, b.turn, weight_list)
print(result)
"""

In [ ]:
"""
Genetic Algorithm to get a weight list.
An agent ought to be represented as a list of weights:
    [mobility, mobility list, static, safety, territory, fortify, kings pawn shield, bishop position, knight positions, hide king, hide queen]
init:
    (Buildable Agent)
    __init__(self, color, norm_method, openingChoice, mobilityWeight, mobilityWeightList, staticAnalysisWeight, safetyWeight, territoryWeight, fortifyWeight, kingsPawnShieldWeights, bishopPositionWeights, knightPositionWeights, hideKingTuple=(0,0), hideQueenTuple=(0,0))
ALGORITHM
Setup:
 -create group of agents with random weights.
 -repeat for the desired number of populations.
 -go to population step
 
Population step: 
 -for each agent in each population, rate it against _ board positions to get its fitness value.
 -take the fittest agent from each population to form the parent pool for the next generation.
 -go to generation step

Generation step:
 -if we've completed the specified number of generations, then take the most fit of the parent pool and return.
 -create a child chromosome using mutation and crossover (see doc)
 -put the child chromosome into a new population
 -repeat until all new populations are filled.
 -go to population step

What aspects can be parallelized:
 -calculating the most fit individual from each population; each population's most fit individual can be done seperately.
 -filling the new populations from the parent pool; each population can be filled seperately.
 -could even parallelize the scoring of individuals in each population; each individual's fitness could be calculated individually
"""
import datetime

def score_fitness(individual, fitness_set):
  """
  Takes an individual and the fitness set
  Returns the individual's fitness (int)
  """
  # we will need a new pick move function, local to here. (the agent's one includes a whole bunch of extra stuff.)
  # takes weight list, and returns best move.

  score = 0
  for i in range(0, len(fitness_set)):
    # move-score process:
    # setup the position
    board = fitness_set[i][0]
 
    # compare to real next move. (which is in san format)
    if individual_select_move(board, board.turn, individual, fitness_set[i][2]) == fitness_set[i][1]:
      score = score + 1

  return score

def population_step(population, fitness_set, mp_queue=None):
  """
  Takes a single population. (list of weight lists)
  Returns the most fit. (weight list)
  """
  runningMax = -1
  bestIndividual = None
  for individual in population:
    score = score_fitness(individual, fitness_set)
    if runningMax == -1 or score > runningMax:
      runningMax = score
      bestIndividual = individual

  if mp_queue is None:
    return bestIndividual
  else:
    mp_queue.put(bestIndividual)

def generation_step(parent_pool, pop_cap, crossover_chance, mutation_chance, mp_queue=None):
  """
  creates a new population based on parents. (list of weight lists)
  Returns a population (list of weight lists)
  """
  new_population = []
  for i in range(0, pop_cap):   
    # select 2 distinct parents
    p1_index, p2_index = random.sample(range(0, len(parent_pool)), 2)
    parent1 = parent_pool[p1_index]
    parent2 = parent_pool[p2_index]

    child = []

    # crossover
    if random.randint(1, 100) < crossover_chance:
      crossover_point = random.randint(0, len(parent1) - 1)
      child = parent1[:crossover_point] + parent2[crossover_point:]
    else:
      if random.choice([0, 1]) == 0:
        child = parent1
      else:
        child = parent2

    # one way or another, child has been assigned.
    # mutation
    for j in range(0, len(child)):
      if random.randint(1, 100) < mutation_chance:
        # perform mutation. 
        # -for now, just randomly regenerate it.
        child[j] = generate_weight(j)
    
    new_population.append(child)
  
  if mp_queue is None:
    return new_population
  else:
    mp_queue.put(new_population)

def run(generation_number, distinct_pops, single_pop_cap, number_of_moves, crossover_chance, mutation_chance):
  """
  Performs the whole procedure.
  """
  fitness_set = assemble_fitness_set(number_of_moves)
  print("finished assembling fitness set")

  world_pop = [] # list of all populations
  
  for i in range(0, distinct_pops):
    world_pop.append([])
    for j in range(0, single_pop_cap):
      world_pop[i].append(create_firstgen_agent())

  parent_pool = []
  # At this point, we have completed setup. Now, run the specified number of generations.
  for i in range(0, generation_number):
    
    # create parent pool by taking the most fit from each population
    parent_pool.clear()

    # print("before population step, worldpop =\n" + str(world_pop))

    """
    parallelized the loop below:
    for j in range(0, distinct_pops):
      parent_pool.append(population_step(world_pop[j], fitness_set))
    """
    output = multiprocessing.Queue()
    processes = []
    for j in range(0, distinct_pops):
      p = multiprocessing.Process(target=population_step, args=(world_pop[j], fitness_set, output))
      processes.append(p)
      p.start()
    for p in processes:
      p.join()
    
    parent_pool = [output.get() for p in processes]
    world_pop.clear()

    # print("after population step, worldpop =\n" + str(world_pop))
    # print("after population step. parent pool =\n" + str(parent_pool))

    # create new populations from parent pool
    """
    parallelized the loop below:
    for j in range(0, distinct_pops):
      world_pop.append(generation_step(parent_pool, single_pop_cap, crossover_chance, mutation_chance))
    """
    output = multiprocessing.Queue()
    processes = []
    for j in range(0, distinct_pops):
      p = multiprocessing.Process(target=generation_step, args=(parent_pool, single_pop_cap, crossover_chance, mutation_chance, output))
      processes.append(p)
      p.start()
    for p in processes:
      p.join()
    
    # print("after generation step. parent pool =\n" + str(parent_pool))
    # print("after generation step, worldpop =\n" + str(world_pop))
    print("generation " + str(i) + " completed @ " + datetime.datetime.now().strftime("%H:%M:%S"))

  # At this point, we have completed all the generations and have the best individuals all stored in the parent pool.
  # return the best individual in the parent pool to end the process.
  best = population_step(parent_pool, fitness_set)
  best_fitness = score_fitness(best, fitness_set)
  return best, best_fitness


In [ ]:
"""
Run the genetic algorithm thing.
Parameters legend:
 -gen_num: the number of generations that will be performed
 -sep_pop_num: the number of distinct populations
 -ind_pop_cap: the number of individuals in each distinct population
 -moves_num: the number of moves each individual is scored against, i.e. the best possible fitness score.

Still to-do:
 -add more games to the dataset. How many should we have total. 100? 50?

The settings used in the paper:
  1000 moves for fitness
  200 generations
  25% crossover rate
  0.7% mutation rate
  10-20 population size (whatever this means, thanks for communicationg clearly)

"""
gen_num = 1  # the number of generations that will occur
sep_pop_num = 5 # the number of distinct populations
ind_pop_cap = 5  # the number of individuals in each population
moves_num = 10  # the number of moves the individual is compared to during fitness scoring

# give as percentage out of 100. 
# e.g. 50 would have a 1/2 chance of occuring.
crossover_chance = 25
mutation_chance = 1

print("Starting @ " + datetime.datetime.now().strftime("%H:%M:%S"))
best_individual, best_ind_fitness = run(gen_num, sep_pop_num, ind_pop_cap, moves_num, crossover_chance, mutation_chance)
print("The best individual is: " + str(best_individual))
print("Their fitness score at the last check was: " + str(best_ind_fitness) + " / " + str(moves_num))


# sanity check:
# score the fitness of the individual against a new random-generated set, to see how overfitted it is.
# (the closer the sanity fitness is to the training fitness the better)
sanity_set = assemble_fitness_set(moves_num)
sanity_fitness = score_fitness(best_individual, sanity_set)
print("Sanity check fitness: " + str(sanity_fitness) + " / " + str(moves_num))

# paper's test:
# 200 generations
# 10 pops
# 10 individuals in each pop
# 1000 moves fitness
# total operations: 200 * 10 * 10 * 1000 = 20 000 000 (20 million)

# our speed for a single operation
# 6 min 15 seconds for: 6 * 20 * 100 = 12 000 (12 thousand)
# -> 375 seconds for 12 000
# -> 0.03125 seconds for 1 individual
# for our speed to do the paper's test would take:
# 0.03125 * 20 000 000 = 625000 seconds
# i.e. 173 hours (LOL)


Starting @ 21:06:40
finished assembling fitness set
generation 0 completed @ 21:06:49
The best individual is: [5, {1: 4, 2: 6, 3: 7, 4: 0, 5: 0, 6: 0}, 4, 1, 1, 4, 0, 9, 1, (0, 4), (4, 3)]
Their fitness score at the last check was: 2 / 10
Sanity check fitness: 0 / 10


Genetic Algorithim Results: (tables are not supported by colab)
1. params=(10, 6, 20, 100, 25, 5) | fitness = 11% | individual = [1, {1: 9, 2: 1, 3: 7, 4: 8, 5: 4, 6: 0}, 2, 6, 1, 5, 8, 6, 3, (6, 10), (19, 4)]
1. params=(10, 6, 20, 100, 25, 1) | fitness = 17% | individual = [4, {1: 7, 2: 0, 3: 8, 4: 6, 5: 9, 6: 0}, 6, 10, 0, 5, 2, 7, 0, (9, 0), (13, 2)]
1. params=(50, 10, 10, 500, 25, 1) | fitness = _% | sanity = _% | individual = []

Below here is the testing (for function correctness) area:

In [ ]:
"""
knight position scoring function tests
"""

pieces1 = {"n1": "c3", "n2": "f6", "p2": "e4", "k": "b2", "b1": "c3", "p5": "b6"}
score1 = knight_positions(pieces1)
assert score1 == 2, f"assert failed. Expected = 2 | score = {score1}"

pieces1 = {"n1": "b3", "n2": "a6", "p2": "b1", "p3": "e4", "p4": "d1", "p5": "f3"}
score1 = knight_positions(pieces1)
assert score1 == 0, f"assert failed. Expected = 0 | score = {score1}"

In [ ]:
"""
bishop position scoring function tests
"""

pieces1 = {"b1": "f6", "b2": "f5", "p2": "b4", "k": "b2", "p1": "c3", "p5": "b6"}
score1 = bishop_positions(pieces1)
assert score1 == 1, f"assert failed. Expected = 1 | score = {score1}"

pieces1 = {"b1": "b3", "b2": "d3", "p2": "b1", "p3": "e4", "p4": "d1", "p5": "f3"}
score1 = bishop_positions(pieces1)
assert score1 == 0, f"assert failed. Expected = 0 | score = {score1}"

In [ ]:
"""
king's pawn shield scoring function tests
"""

pieces1 = {"k": "b3", "p1": "a3", "p2": "b4", "p3": "b2", "p4": "c3", "p5": "b6"}
score1 = kings_pawn_shield(pieces1)
assert score1 == 4, f"assert failed. Expected = 4 | score = {score1}"

pieces1 = {"k": "b3", "p1": "a6", "p2": "b1", "p3": "e4", "p4": "d1", "p5": "f3"}
score1 = kings_pawn_shield(pieces1)
assert score1 == 0, f"assert failed. Expected = 0 | score = {score1}"

In [ ]:
"""
Fortify Analysis Test Cases.
fortify_analysis(board, piecesDict, color)

"""

"""
Case 1: starting position.
"""
board = chess.Board()

board.push(chess.Move.from_uci("b2b4"))

w1_dict = {"p1": "a2","p2": "b4","p3": "c2","p4": "d2","p5": "e2","p6": "f2","p7": "g2","p8": "h2","r1": "a1","r2": "h1","n1": "b1","n2": "g1","b1": "c1","b2": "f1","q": "d1","k": "e1"}
w1_score = fortify_analysis(board.copy(), w1_dict, True)
assert w1_score == 10, f"w1_score assert failed. Expected = 10 | w1_score = {w1_score}"

b1_dict = {"p1": "a7","p2": "b7","p3": "c7","p4": "d7","p5": "e7","p6": "f7","p7": "g7","p8": "h7","r1": "a8","r2": "h8","n1": "b8","n2": "g8","b1": "c8","b2": "f8","q": "d8","k": "e8"}
b1_score = fortify_analysis(board.copy(), b1_dict, False)
assert b1_score == 11, f"b1_score assert failed. Expected = 11 | b1_score = {b1_score}"


"""
Case 2: A few moves along.
"""
move_seq = ["e7e6", "g2g3", "b8a6", "f1g2", "c7c5", "g1f3", "f8d6", "d2d4"]
for move in move_seq:
    board.push(chess.Move.from_uci(move))

w2_dict = {"p1": "a2","p2": "b4","p3": "c2","p4": "d4","p5": "e2","p6": "f2","p7": "g3","p8": "h2","r1": "a1","r2": "h1","n1": "b1","n2": "f3","b1": "c1","b2": "g2","q": "d1","k": "e1"}
w2_score = fortify_analysis(board, w2_dict, True)
assert w2_score == 11, f"w2_score assert failed. Expected = 11 | w2_score = {w2_score}"

b2_dict = {"p1": "a7","p2": "b7","p3": "c5","p4": "d7","p5": "e6","p6": "f7","p7": "g7","p8": "h7","r1": "a8","r2": "h8","n1": "a6","n2": "g8","b1": "c8","b2": "d6","q": "d8","k": "e8"}
b2_score = fortify_analysis(board, b2_dict, False)
assert b2_score == 10, f"b2_score assert failed. Expected = 10 | b2_score = {b2_score}"


In [ ]:
"""
(Note: not updated to new version of mobility function)

Mobility Test Cases.

piece_weights = {
        1 : 1, # pawn
        2 : 3, # knight
        3 : 3, # bishop
        4 : 5, # rook
        5 : 9, # queen
        6 : 0  # king
    }
WHITE:
w_dict = {"p1": "a2","p2": "b2","p3": "c2","p4": "d2","p5": "e2","p6": "f2","p7": "g2","p8": "h2","r1": "a1","r2": "h1","n1": "b1","n2": "g1","b1": "c1","b2": "f1","q": "d1","k": "e1"}
BLACK:
b_dict = {"p1": "a7","p2": "b7","p3": "c7","p4": "d7","p5": "e7","p6": "f7","p7": "g7","p8": "h7","r1": "a8","r2": "h8","n1": "b8","n2": "g8","b1": "c8","b2": "f8","q": "d8","k": "e8"}
"""

"""
Case 1: No moves yet made.
"""
board = chess.Board()
w1_dict = {"p1": "a2","p2": "b2","p3": "c2","p4": "d2","p5": "e2","p6": "f2","p7": "g2","p8": "h2","r1": "a1","r2": "h1","n1": "b1","n2": "g1","b1": "c1","b2": "f1","q": "d1","k": "e1"}
b1_dict = {"p1": "a7","p2": "b7","p3": "c7","p4": "d7","p5": "e7","p6": "f7","p7": "g7","p8": "h7","r1": "a8","r2": "h8","n1": "b8","n2": "g8","b1": "c8","b2": "f8","q": "d8","k": "e8"}
w1_score = mobility_analysis(board, w1_dict, True)
b1_score = mobility_analysis(board, b1_dict, False)
assert w1_score == 28, f"w1_score assert failed. Expected = 28 | w1_score = {w1_score}"
assert b1_score == 28, f"b1_score assert failed. Expected = 28 | b1_score = {b1_score}"

"""
Case 2: London System vs Bongcloud. 
"""
move_seq = ["d2d4", "e7e5", "g1f3", "e8e7", "c1f4"]
for move in move_seq:
    board.push(chess.Move.from_uci(move))
w2_dict = {"p1": "a2","p2": "b2","p3": "c2","p4": "d4","p5": "e2","p6": "f2","p7": "g2","p8": "h2",
           "r1": "a1","r2": "h1","n1": "b1","n2": "f3","b1": "f4","b2": "f1","q": "d1","k": "e1"}
b2_dict = {"p1": "a7","p2": "b7","p3": "c7","p4": "d7","p5": "e5","p6": "f7","p7": "g7","p8": "h7",
           "r1": "a8","r2": "h8","n1": "b8","n2": "g8","b1": "c8","b2": "f8","q": "d8","k": "e7"}
w2_score = mobility_analysis(board, w2_dict, True)
b2_score = mobility_analysis(board, b2_dict, False)
assert w2_score == 91, f"w2_score assert failed. Expected = 91 | w2_score = {w2_score}"
assert b2_score == 38, f"b2_score assert failed. Expected = 38 | b2_score = {b2_score}"

"""
Case 3: Englund Gambit declined: Reversed Alekhine Variation
"""
move_seq_2 = ["e5f4", "g2g3", "b7b5", "b1c3", "c8b7", "g3f4", "d7d5", "c3d5", "d8d5", "d1d3"]
for move in move_seq_2:
    board.push(chess.Move.from_uci(move))
w3_dict = {"p1": "a2","p2": "b2","p3": "c2","p4": "d4","p5": "e2","p6": "f2","p7": "f4","p8": "h2",
           "r1": "a1","r2": "h1","n2": "f3","b2": "f1","q": "d3","k": "e1"}
b3_dict = {"p1": "a7","p2": "b5","p3": "c7","p6": "f7","p7": "g7","p8": "h7",
           "r1": "a8","r2": "h8","n1": "b8","n2": "g8","b1": "b7","b2": "f8","q": "d5","k": "e7"}
w3_score = mobility_analysis(board, w3_dict, True)
b3_score = mobility_analysis(board, b3_dict, False)
assert w3_score == 160, f"w3_score assert failed. Expected = 160 |  w3_score = {w3_score}"
assert b3_score == 179, f"b3_score assert failed. Expected = 179 |  b3_score = {b3_score}"


In [ ]:
"""
Static Analysis Test Cases.

def staticAnalysis(board, my_color)

piece values:
(chess.PAWN, 1)
(chess.BISHOP, 4)
(chess.KING, 0)
(chess.QUEEN, 10)
(chess.KNIGHT, 5)
(chess.ROOK, 3)
total = 8 + 8 + 0 + 10 + 10 + 6 = 42
"""

"""
Case 1: No moves yet made.
"""
board = chess.Board()
w1_score = staticAnalysis(board, True)
b1_score = staticAnalysis(board, False)
assert w1_score == 41, f"w1_score assert failed. Expected = 41 | w1_score = {staticAnalysis(board, True)}"
assert b1_score == 41, f"b1_score assert failed. Expected = 41 | b1_score = {staticAnalysis(board, False)}"

"""
Case 2: black loses a pawn, white loses its queen.
"""
move_seq = ["c2c4", "g7g5", "d1c2", "a7a6", "c2h7", "h8h7"]
for move in move_seq:
    board.push(chess.Move.from_uci(move))
w2_score = staticAnalysis(board, True)
b2_score = staticAnalysis(board, False)
assert w2_score == 32, f"w2_score assert failed. Expected = 32 | w2_score = {staticAnalysis(board, True)}"
assert b2_score == 40, f"b2_score assert failed. Expected = 01 | b2_score = {staticAnalysis(board, False)}"


In [ ]:
"""
Safety Analysis Test Cases.

def safety_analysis(board, piece_dict)

Note: this scoring function only makes sense in the following circumstances:
- player X has just moved.
- call safety_analysis(updated_board, X_piece_dictionary)
- this will return X's safety score.
Bref, call it for the inactive player.
"""

"""
Case 1: Perfect safety.
"""
board = chess.Board()
b1_dict = {"p1": "a7","p2": "b7","p3": "c7","p4": "d7","p5": "e7","p6": "f7","p7": "g7","p8": "h7","r1": "a8","r2": "h8","n1": "b8","n2": "g8","b1": "c8","b2": "f8","q": "d8","k": "e8"}
b1_score = safety_analysis(board, b1_dict)
assert b1_score == 16, f"b1_score assert failed. Expected = 16 | b1_score = {safety_analysis(board, False)}"

board.push(chess.Move.from_uci("e2e4"))

w1_dict = {"p1": "a2","p2": "b2","p3": "c2","p4": "d2","p5": "e2","p6": "f2","p7": "g2","p8": "h2","r1": "a1","r2": "h1","n1": "b1","n2": "g1","b1": "c1","b2": "f1","q": "d1","k": "e1"}
w1_score = safety_analysis(board, w1_dict)
assert w1_score == 16, f"w1_score assert failed. Expected = 16 | w1_score = {safety_analysis(board, True)}"


"""
Case 2: A few moves along, and danger is creeping in.
"""
move_seq = ["c7c5", "f1b5", "g8f6", "d1f3", "h7h5", "b2b3", "e7e6", "c1b2"]
for move in move_seq:
    board.push(chess.Move.from_uci(move))

w2_dict = {"p1": "a2","p2": "b3","p3": "c2","p4": "d2","p5": "e4","p6": "f2","p7": "g2","p8": "h2","r1": "a1","r2": "h1","n1": "b1","n2": "g1","b1": "b2","b2": "b5","q": "f3","k": "e1"}
w2_score = safety_analysis(board, w2_dict)
assert w2_score == 15, f"w2_score assert failed. Expected = 15 | w2_score = {safety_analysis(board, w2_dict)}"

board.push(chess.Move.from_uci("b8a6"))

b2_dict = {"p1": "a7","p2": "b7","p3": "c5","p4": "d7","p5": "e6","p6": "f7","p7": "g7","p8": "h5","r1": "a8","r2": "h8","n1": "a6","n2": "f6","b1": "c8","b2": "f8","q": "d8","k": "e8"}
b2_score = safety_analysis(board, b2_dict)
assert b2_score == 12, f"b2_score assert failed. Expected = 12 | b2_score = {safety_analysis(board, b2_dict)}"


In [ ]:
"""
Territory Test Cases.

def territory_analysis(board, dummyMove)

scenario:
 - we want to score white's position
 - it's black's turn
 - call territory_analysis(board, white_pieces, black_dummy_move)
    - we have black perform a dummy move so as not to change the game state
    - white's turn again; 
    - return number of legal moves that end on unique positions

"""

"""
Case 1: starting position.
"""
board = chess.Board()

board.push(chess.Move.from_uci("b2b4"))


w1_score = territory_analysis(board.copy())
assert w1_score == 16, f"w1_score assert failed. Expected = 16 | w1_score = {w1_score}"

b1_score = territory_analysis(board.copy())
assert b1_score == 16, f"b1_score assert failed. Expected = 16 | b1_score = {b1_score}"



"""
Case 2: A few moves along.
"""
move_seq = ["e7e6", "g2g3", "b8a6", "f1g2", "c7c5", "g1f3", "f8d6", "d2d4"]
for move in move_seq:
    board.push(chess.Move.from_uci(move))


w2_score = territory_analysis(board)
assert w2_score == 21, f"w2_score assert failed. Expected = 21 | w2_score = {w2_score}"

b2_score = territory_analysis(board)
assert b2_score == 20, f"b2_score assert failed. Expected = 20 | b2_score = {b2_score}"





Cold storage (unused functions, but they may come in handy, so keep them around, okay~)

In [ ]:
"""
Cold Storage.
"""

def old_mobility_unused_stuff():
    # print(f"positional_analysis(). color = {color}. pieces_dict={piecesDictionary}")
    letterToX = {"a" : 1,"b" : 2,"c" : 3,"d" : 4,"e" : 5,"f" : 6,"g" : 7,"h" : 8}
    xToLetter = {1 : "a",2 : "b",3 : "c",4 : "d",5 : "e",6 : "f",7 : "g",8 : "h"}

    def isValidCoords(pos):
      # takes pos, a tuple (x, y) as an argument.
      # returns true if the position is within dimensions of a chess board. returns false otherwise.
      if pos[0] < 9 and pos[0] > 0 and pos[1] < 9 and pos[1] > 0:
          return True
      return False

    def isValidPieces(pos):
      # returns true if there is not an ally piece on the pos.
      chessPos = xToLetter[pos[0]] + str(pos[1])
      if board.piece_at(chess.SQUARE_NAMES.index(chessPos)) is None or board.piece_at(chess.SQUARE_NAMES.index(chessPos)).color != color:
        return True
      return False

    def isEnemyCoords(pos):
      # returns true if there is an enemy piece on the pos.
      chessPos = xToLetter[pos[0]] + str(pos[1])
      if board.piece_at(chess.SQUARE_NAMES.index(chessPos)) is not None and board.piece_at(chess.SQUARE_NAMES.index(chessPos)).color != color:
        return True
      return False

    # the movement calculating functions. Returns the number of moves, times the piece's weight.
    def pawn(pos):
      # if the tile in front (check color for direction) of the pawn is empty, return 1. Otherwise, return 0.
      # given (x,y), if there is a piece at x-1, y+1 or x+1, y+1, add that too.
      # does not consider en passant. Sorry.

      # determine the code for the spot
      x = int(letterToX[pos[0:1]])
      y = int(pos[1:])

      diagonalSpots = []
      if color: # if white, then we can move in a way that increases our y.
        checkSpot = (x, y+1)
        if isValidCoords((x-1, y+1)):
          diagonalSpots.append((x-1, y+1))     
        if isValidCoords((x+1, y+1)):
          diagonalSpots.append((x+1, y+1))
      else: # if black, then we can move in a way that decreases our y.
        checkSpot = (x, y-1)
        if isValidCoords((x-1, y-1)):
          diagonalSpots.append((x-1, y-1))     
        if isValidCoords((x+1, y-1)):
          diagonalSpots.append((x+1, y-1))

      # checkSpot is valid if there is no piece of any side there.
      sum = 0
      chessPos = chess.SQUARE_NAMES.index(xToLetter[checkSpot[0]] + str(checkSpot[1]))
      if isValidCoords(checkSpot) and board.piece_at(chessPos) is None:
        sum += 1
      # diagonalSpots are valid if there is an enemy piece there
      for i in range(0, len(diagonalSpots)):
        chessPos = chess.SQUARE_NAMES.index(xToLetter[diagonalSpots[i][0]] + str(diagonalSpots[i][1]))
        if isValidCoords(diagonalSpots[i]) and board.piece_at(chessPos) is not None and board.piece_at(chessPos).color != color:
          sum += 1
      return sum

    def knight(pos):
      # the knight has at most 8 possible moves.

      # determine the codes for the spot. Use math to include only moves that are within the board.
      x = letterToX[pos[0:1]]
      y = int(pos[1:])

      checkSpots = [(x-1, y+2), (x+1, y+2), (x-2, y+1), (x+2, y+1), (x-2, y-1), (x+2, y-1), (x-1, y-2), (x+1, y-2)]
      sum = 0
      for i in range(0, len(checkSpots)):
        # if move is within the board
        if isValidCoords(checkSpots[i]) and isValidPieces(checkSpots[i]):
          sum += 1
      return sum

    def rook(pos):
      # the rook can move in 4 directions, and at most 7 in any direction.
      # only one of its coordinates can change in any move.
      x = letterToX[pos[0:1]]
      y = int(pos[1:])

      # count x moves. Can continue until blocked by a piece. (if allied piece, do not count blocked move. If opposing, do count.)
      
      # we need to explore 4 paths:
      # x++++, y
      # x----, y
      # x, y++++
      # x, y----
      sum = 0

      # move to the left. Explore from (x, y) to (0, y)
      for i in range (x - 1, 0, -1):
        spot = (i, y)
        # if the spot exists on the board and there is no ally on the spot
        if isValidCoords(spot) and isValidPieces(spot):
              sum += 1
              if isEnemyCoords(spot):
                break
        else:
          break
      # move to the right. Explore from (x, y) to (8, y)
      for i in range (x + 1, 8+1):
        spot = (i, y)
        # if the spot exists on the board and there is no ally on the spot
        if isValidCoords(spot) and isValidPieces(spot):
              sum += 1
              if isEnemyCoords(spot):
                break
        else:
          break
      # move to the bottom. Explore from (x, y) to (x, 0)
      for i in range (y - 1, 0, -1):
        spot = (x, i)
        # if the spot exists on the board and there is no ally on the spot
        if isValidCoords(spot) and isValidPieces(spot):
              sum += 1
              if isEnemyCoords(spot):
                break
        else:
          break
      # move to the top. Explore from (x, y) to (x, 8)
      for i in range (y + 1, 8+1):
        spot = (x, i)
        # if the spot exists on the board and there is no ally on the spot
        if isValidCoords(spot) and isValidPieces(spot):
              sum += 1
              if isEnemyCoords(spot):
                break
        else:
          break
      return sum

    def bishop(pos):
      debug_pos = ""
      # the bishop can move in 4 diagonal directions, and at most 7 in any direction.
      # its x and y change in any possible move.
      x = letterToX[pos[0:1]]
      y = int(pos[1:])

      sum = 0
      # move to the top left. Explore from (x, y) to (0, 8)
      i = x
      j = y
      while i > 0 and j < 9:
        i -= 1
        j += 1
        spot = (i, j)
        if isValidCoords(spot) and isValidPieces(spot):
              if pos == debug_pos:
                  print(f"adding {spot}")
              sum += 1
              if isEnemyCoords(spot):
                break
        else:
          break
      # move to the bottom left. Explore from (x, y) to (0, 0)
      i = x
      j = y
      while i > 0 and j > 0:
        i -= 1
        j -= 1
        spot = (i, j)
        if isValidCoords(spot) and isValidPieces(spot):
              if pos == debug_pos:
                      print(f"adding {spot}")
              sum += 1
              if isEnemyCoords(spot):
                break
        else:
          break
      # move to the top right. Explore from (x, y) to (8,8)
      i = x
      j = y
      while i < 9 and j < 9:
        i += 1
        j += 1
        spot = (i, j)
        if isValidCoords(spot) and isValidPieces(spot):
              if pos == debug_pos:
                  print(f"adding {spot}")
              sum += 1
              if isEnemyCoords(spot):
                break
        else:
          break
      # move to the bottom right. Explore from (x, y) to (8,0)
      i = x
      j = y
      while i < 9 and j > 0:
        i += 1
        j -= 1
        spot = (i, j)
        if isValidCoords(spot) and isValidPieces(spot):
              if pos == debug_pos:
                  print(f"adding {spot}")
              sum += 1
              if isEnemyCoords(spot):
                break
        else:
          break

      return sum

    def queen(pos):
      # what is a queen but a bishop and a rook in a single piece? (rhetorical)
      return rook(pos) + bishop(pos)

    def king(pos):
      # by convention, many mobility scoring methods ignore the king. Therefore.
      return 0

    options = {
        1 : pawn,
        2 : knight,
        3 : bishop,
        4 : rook,
        5 : queen,
        6 : king
    }

    sum = 0

    if not debugging:
        for key, pos in piecesDictionary.items():
            piece = board.piece_at(chess.SQUARE_NAMES.index(pos))
            # print(f"MOB: evaluating piece {key} at {pos}")
            if piece is None:
              # error: print out board.
              print(f"key: {key}, pos: {pos}")
              print("Error. Printing out board:")
              html_code = chess.svg.board(board, size=300)
              display(IPython.display.HTML(html_code))

            piece_type = piece.piece_type
            # use piece_type to call the right helper function and pos as the argument.
            sum += (options[piece_type](pos) * piece_weights[piece_type])
        return sum
    else:
        print("======START======")
        html_code = chess.svg.board(board, size=300)
        display(IPython.display.HTML(html_code))
        for key, pos in piecesDictionary.items():
              piece = board.piece_at(chess.SQUARE_NAMES.index(pos))
              # print(f"MOB: evaluating piece {key} at {pos}")
              piece_type = piece.piece_type
              score = (options[piece_type](pos) * piece_weights[piece_type])
              print(f"key: {key}, pos: {pos}, score: {score}")
              sum += score
        print("=======END=======")
        return sum
